In [1]:
import os
print(f"현재 PID: {os.getpid()}")

현재 PID: 3616504


In [ ]:
# 베스트 모델 시드 고정

# 전체코드: ResNet18 + CBAM + MGA Loss + Lambda Scheduling (CE Weight ver)

import os, re, numpy as np, torch, gc
import csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import pandas as pd
import cv2
import torchvision.transforms as transforms
from PIL import Image
from datetime import datetime
import random


# -------------------- 디바이스 설정 --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------- 하이퍼파라미터 설정 --------------------
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"

batch_size = 16
num_epochs = 100
learning_rate = 1e-4
seed_list = [42, 123, 2025, 777, 999]
all_metrics = []

# lambda MGA 스케줄 설정
initial_lambda = 0.1
final_lambda = 0.5
total_epochs = num_epochs

# -------------------- Transform --------------------
train_transform = transforms.Compose([
    transforms.ToPILImage(),    # numpy or tensor 이미지를 PIL 이미지 객체로 변환
    transforms.Resize((224, 224)),  # 이미지를 224x224로 resize
    transforms.RandomHorizontalFlip(),  # 이미지를 50% 확률로 좌우 반전
    transforms.RandomRotation(10),  # 이미지를 -10도 ~ +10도 사이로 랜덤 회전, 촬영 자세나 기울어짐에 대한 회전 강건성확보
    transforms.ToTensor(),  # PIL이미지 -> PyTorch Tensor로 변환, (H, W, C) -> (C, H, W), 값도 0255 -> 01 사이즈로 스케일 조정
    transforms.Normalize([0.5], [0.5]), # 평균 0.5, 표준편차 0.5로 정규화 -> 결과적으로 01 -> 11로 바뀜
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])  # 전체 이미지의 일부분 지우고 0으로 채움 (검은 사각형 생김)
    # p=0.5 : 50% 확률로 이 증강 적용
    # scale : 전체 이미지 대비 삭제 영역의 크기 비율
    # ratio : 지우는 사각형의 가로:세로 비율 범위
    # value=0 : 지운 곳을 검은색(0)으로 덮음
    # 폐 CT에서 병변이 항상 일정한 위치에 나오지 않으니까 모델이 특정 위치에 과적합되는걸 방지함 (overfitting 예방)

# 검증/테스트는 모델이 학습하지 않은 깨긋한 상태의 이미지로 정확도를 확인하기 위해서 검증용은 깔끔하게
val_transform = transforms.Compose([
    transforms.ToPILImage(),    # PIL 이미지 객체로 변환
    transforms.Resize((224, 224)),  # 사이즈 맞추기
    transforms.ToTensor(),  # PIL 이미지 -> PyTorch Tensor로 변환
    transforms.Normalize([0.5], [0.5])  # 정규화
])

# 시드 고정
def seed_everything(seed=42):
    random.seed(seed)                         # 파이썬 random
    np.random.seed(seed)                      # numpy
    torch.manual_seed(seed)                   # torch CPU
    torch.cuda.manual_seed(seed)              # torch GPU
    torch.cuda.manual_seed_all(seed)          # multi-GPU
    torch.backends.cudnn.deterministic = True # 연산 동일하게
    torch.backends.cudnn.benchmark = False    # 연산 속도 최적화 OFF (같은 연산 보장)

# -------------------- Bounding Box를 Binary Mask로 --------------------
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    # bbox_list : 한 이미지에 들어있는 bounding box 리스트
    # image_size : 출력할 마스크 크기. 보통 이미지와 동일한 (height, width) -> 디폴트는 224x224
    # bbox들을 binary mask로 바꿔주는 함수
    masks = []  # 여러 개의 bbox가 들어오니까, 각각의 마스크를 하나씩 리스트에 쌓기 위한 빈 리스트
    for bbox in bbox_list:  # bbox_list를 하나씩 돌면서 처리 -> [x_min, y_min, x_max, y_max]네 좌표로 구성된 하나의 사각형 영역 
        mask = np.zeros(image_size, dtype=np.float32)   # 224x224짜리 0으로 꽉 찬 2D 배열을 하나 생성
        # 배경이 흰 종이를 만드는 느낌으로 만들고, 사각 영역만 1로 덧칠할거임
        x_min, y_min, x_max, y_max = bbox   # 각 bbox의 네 좌표값을 각각 변수로 언팩. -> 마스크의 해당 영역에 사각형을 칠하기 위해서
        mask[y_min:y_max, x_min:x_max] = 1.0    # y_min, y_max, x_min, x_max까지의 범위에 1.0을 채워 넣음
        # -> 마스크에서 bbox에 해당하는 사각형 영역만 1(foreground)로 표시됨. 나머진 여전히 0(background)
        masks.append(mask)  # 지금 만든 마스크(2D 배열)를 리스트에 추가 -> [mask1, mask2, ...]이렇게 쌓임

    masks = np.stack(masks) # 리스트를 하나의 3D 배열로 합침 -> shape : [N, H, W] -> N은 bbox 개수
    masks = np.expand_dims(masks, axis=1)   # 텐서 shape을 [N, 1, H, W]로 바꿈
    # PyTorch 모델에서 기대하는 (batch x channel x height x width) 포맷 맞추기

    return torch.tensor(masks, dtype=torch.float32)
    # numpy 배열을 PyTorch 텐서로 변환해서 리턴

    # 한 bbox → 하나의 마스크 → 여러 개면 쌓아서 batch 형태로
# -------------------- Bounding Box CSV 로드 --------------------
def load_bbox_dict(csv_path):
    # csv_path : bounding box 정보가 들어있는 CSV 파일 경로
    # 반환값 : {filename:[bbox1, bbox2, ...]} 형태의 딕셔너리
    df = pd.read_csv(csv_path)  # CSV파일을 pandas DataFrame으로 읽어옴
    bbox_dict = {}
    # key : 슬라이스 파일 이름 (ex. "LIDC-IDRI-1012_slice0004.npy")
    # value : 해당 슬라이스에 존재하는 bbox들의 리스트
    for _, row in df.iterrows():    # DataFrame의 모든 행(row)를 하나씩 순회
        # row는 한 줄(=한 bbox)의 정보를 담고 있음

        pid = row['pid']    # 환자 ID (예: "LIDC-IDRI-1012") -> 이미지 이름 구성 요소
        slice_str = row['slice']    # 슬라이스 정보가 들어있는 문자열 (예: "slice_0039")
        slice_idx = int(re.findall(r'\d+', str(slice_str))[0])  # re.findall()로 문자열에서 숫자만 뽑아냄
        # "slice_0039" -> ['0039'] -> [0] -> 39 (슬라이스 번호를 정수로 추출함)
        fname = f"{pid}_slice{slice_idx:04d}.npy"   # 파일명 구성 (예: "LIDC-IDRI-1012_slice0039.npy")
        # {:04d}는 4자리 정수로 만들고 빈자리는 0으로 채워줌 (39 -> 0039)
        bbox = eval(row['bb'])  # row['bb']는 문자열 형태의 bbox (예: "[20, 30, 80, 100]")
        # eval()을 써서 문자열을 리스트로 바꿔줌
        # 주의 : 보안 상 위험할 수 있는 함수지만, 여긴 내부 데이터라 사용중
        bbox_dict.setdefault(fname, []).append(bbox)    # fname이라는 key가 딕셔너리에 없으면 []로 초기화하고,
        # 거기에 bbox를 append -> 슬라이스 하나에 bbox 여러개 있어도 전부 리스트로 모아줌
    return bbox_dict    # 최종적으로 {filename: [bbox1, bbox2, ...]} 형태의 딕셔너리 반환

bbox_dict = load_bbox_dict(bbox_csv_path)
# 실제로 csv_path에 있는 정보를 불러와서 bbox_dict에 저장함
# 이걸 나주에 Dataset 클래스에서 fname 기준을 꺼내쓰게 됨

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname): # fname : 파일 이름 (예: "LIDC-IDRI-1012_slice0039_5.npy")
    # 이 이름에서 malignancy score(악성도 점수)를 추출해서 라벨로 변환
    try:    # 파일명이 이상하거나 에러나면 except로 빠져나가서 None 반환함 (안전장치)
        score = int(fname.split("_")[-1].replace(".npy", ""))
        # 파일명에서 _ 제외하고 나머지 것들 중에 마지막에꺼를 가져와서 .npy를 "" 이렇게 공백으로 처리함
        # fname.split("_") -> ['LIDC-IDRI-1012', 'slice0039', '5.npy]
        # [-1] -> '5.npy'
        # .replace(".npy", "") -> '5'
        # int(...) -> 5 <- 이게 malignancy score
        return None if score == 3 else int(score >= 4)
        # 라벨 결정 로직으로
        # score == 3 -> 중립 -> None 반환 -> 학습에서 제외
        # score >= 4 -> 암(양성) -> 1
        # score <= 2 -> 정상(음성) -> 0
        # int(score >= 4)는 파이썬에서 True -> 1
        # False -> 0 이니깐 자동으로 라벨이 됨
    except:
        return None
        # 혹시 split이나 replace, int 변환이 실패하면 그냥 None 반환하고 무시

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    # PyTorch의 Dataset 클래스를 상속해서 커ㅡ텀 데이터셋 정의
    # 나중에 DataLoader랑 같이 쓰이기 때문에 __len__()이랑 __getitem__()을 꼭 넣어줘야함
    def __init__(self, paths, labels, transform=None):  # 생성자 : 세개의 인자를 받음
        # paths : 이미지 .npy 파일 경로 리스트
        # labels : 각 이미지에 대한 라벨 리스트 (0, 1 or None)
        # transform : 이미지 증강 설정 (train_transform, val_transform 등)
        self.paths = paths
        self.labels = labels
        self.transform = transform
        # 받은 인자를 멤버 변수로 저장. 나중에 gettem()에서 접근함

    def __getitem__(self, idx): # DataLoader가 이걸 호출할 때 index에 해당하는 sample 하나를 반환
        # 이미지, 라벨, 마스크( = MGA용 target) 3개를 리턴함
        file_path = self.paths[idx] # 파일 경로 불러오기
        label = self.labels[idx]    # 라벨 불러오기
        fname = os.path.basename(file_path) # 전체 경로에서 파일 이름만 추출 -> 나중에 bbox_dict[fname] 찾을때 쓰임

        img = np.load(file_path)    # .npy 파일에서 CT 슬라이스 불러오기 -> 흑백 CT 이미지, shape은 (H, W)
        img = np.clip(img, -1000, 400)  # CT 이미지 HU 값이 너무 크거나 작으면 노이즈 -> -1000(공기) ~ 400(연조직)으로 클리핑해서 노이즈 제거
        img = (img + 1000) / 1400.  # 정규화 : -1000 -> 0, 400 -> 1 사이 값으로 바꿔줌 -> 모델이 안정적으로 학습할 수 있도록 함
        img = np.expand_dims(img, axis=-1)  # CT는 채널이 1개니깐 (H, W) -> (H, w, 1)로 바꿔줌
        # 나중에 PyTorch에서 (C, H, W)로 바꾸기 위함

        if self.transform:  # 데이터 증강(transform)이 있다면 적용
            img = self.transform(img)   
        else:   # 없으면 numpy -> tensor 변환하고 (H, W, C) -> (C, H, W)로 순서 바꿈
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)

        if fname in bbox_dict:  # 이 이미지에 bbox가 존재하면 -> 마스크 생성
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(224, 224))
            # image_size는 transform과 동일하게 224x224
        else:   # bbox가 없다면 전부 0으로 채워진 마스크 생성 -> MGA Loss 계산 시 참고용으로 쓰일 수 있음
            mask = torch.zeros((1, 224, 224), dtype=torch.float32)

        return img, torch.tensor(label).long(), mask.squeeze(0)
        # 반환값 3개 :
        # img : shape[1, 224, 224]
        # label : int(0 or 1)
        # mask : [224, 224] <- squeeze로 채널 1개 제거

    def __len__(self):
        return len(self.paths)
    # 전체 데이터셋 길이 반환 -> DataLoader가 아라야 배치 쪼갤 수 있음.

# -------------------- CBAM 정의 (MGA 포함) --------------------
# 2 Step : Channel Attention(어떤 채널에 집중할지) * Spatial Attention(어디에 집중할지) = 최종 Attention

class ChannelAttention(nn.Module):  # 입력 feature map의 채널별 중요도를 계산해서 강조함
    def __init__(self, planes, ratio=16):
        # planes : 입력 채널 수
        # ratio : 중간 채널 축소 비율. 기본 1/16으로 bottlenck 구성
        super().__init__()

        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        # MLP 역할을 하는 1x1 conv 블록 -> 채널 압축 -> 비선형 -> 복원 (shared는 avg/max 둘다에서 같이 씀)

        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()
        # 평균 풀링 / 최대 풀링으로 두가지 전역 정보를 추출
        # 마지막 sigmoid는 attention weight로 스케일링

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))
    # avg & max 풀링 경과를 각각 shape MLP에 통과시키고, 더한 후 sigmoid
    # -> shape : [B, C, 1, 1]
    # -> 채널마다 중요도 weight를 곱하게 됨

class SpatialAttention(nn.Module):  # 공간적으로 어디에 집중할지를 결정 -> 각 채널 내부에서 중요한 위치 찾기

    def __init__(self, k=7):    
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    # 채널 차원은 평균, 최대 두 개만 써서 concat
    # 그걸 1채널로 줄여주는 conv
    # 커널 크기 k=7이면 넓은 영역까지 감지 가능

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))
    # 입력 feature map에서 :
    # 평균, 최대값을 각 spatial 위치별로 구함 -> [B, 1, H, W] 두 개
    # concat -> [B, 2, H, w]
    # conv + sigmoid -> 위치별 중요도 map

class CBAM(nn.Module):  
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()
        self.last_attention = None
    # ChannelAttention, SpatialAttention을 내부에 선언
    # MGA를 위해 마지막 attention map을 저장하는 변수 포함

    def forward(self, x):
        ca_out = self.ca(x) * x
        sa_out = self.sa(ca_out)
        self.last_attention = sa_out
        return sa_out * ca_out
    # 채널 중요도 -> 곱함
    # 위치 중요도 -> 곱함
    # 둘 다 반영된 최종 feature map 리턴

# -------------------- ResNet18 + CBAM 모델 정의 --------------------
# BasicBlockCBAM : ResNet의 기본 Residual Block 하나를 정의
# → conv → BN → ReLU → conv → BN → (CBAM optional) → Add → ReLU

# ResNet18_CBAM : ResNet18 구조로 전체 네트워크 쌓기
# → conv1 → layer1~3 → layer4 → avgpool → fc

class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        # 입력 채널: in_planes, 출력 채널: out_planes, 3x3 커널, padding=1로 크기 유지, stride로 크기 조절
        self.bn1 = nn.BatchNorm2d(out_planes)   # 배치 정규화
        self.relu = nn.ReLU()   # 비선형 활성화 함수

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        # 두번째 conv, 채널 수 유지, 크기 유지
        self.bn2 = nn.BatchNorm2d(out_planes)   # 배치 정규화

        self.cbam = CBAM(out_planes) if use_cbam else None  # CBAM 모듈 사용 여부
        self.downsample = downsample    # residual 연결 시 차원 맞추는 conv

    def forward(self, x):
        residual = x    # skip connection용 입력 저장

        out = self.conv1(x) # 첫 번째 conv
        out = self.bn1(out) # 정규화
        out = self.relu(out)  # 활성화

        out = self.conv2(out)   # 두 번째 conv
        out = self.bn2(out) # 정규화

        if self.cbam:
            out = self.cbam(out)    # CBAM 적용

        if self.downsample:
            residual = self.downsample(x)   # shortcut 경로 보정

        out += residual # skip connection
        out = self.relu(out)    # 출력에 ReLU 적용

        return out  # 결과 반환

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64 # 조기 입력 채널 수 설정

        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # 입력: [B, 1, 224, 224] -> 출력: [B, 64, 112, 112], 큰 커널로 넓은 영역 캡처 
        self.bn1 = nn.BatchNorm2d(64)   # 정규화
        self.relu = nn.ReLU()   # 활성화

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # 풀링: [B, 64, 112, 112] -> [B, 64, 56, 56]

        self.layer1 = self._make_layer(64, blocks=2)  # [B, 64, 56, 56] 유지
        self.layer2 = self._make_layer(128, blocks=2, stride=2)  # [B, 128, 28, 28] 다운샘플링
        self.layer3 = self._make_layer(256, blocks=2, stride=2)  # [B, 256, 14, 14] 다운샘플링
        self.layer4 = self._make_layer(512, blocks=2, stride=2, use_cbam=False)  # [B, 512, 7, 7], CBAM 미사용

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # [B, 512, 7, 7] -> [B, 512, 1, 1]
        self.fc = nn.Linear(512, num_classes)  # [B, 512] -> [B, num_classes]

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        # Planes : 해당 레이어의 출력 채널 수
        # # blocks : 블록 수
        # stride=2인 경우 다운샘플링 (해상도 절반)

        downsample = None   # 스킵 연결해서 입력/출력 크기가 다르면 맞춰야 함

        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                # 1x1 conv로 채널 수   및 공간 크기 맞춤
                nn.BatchNorm2d(planes))
            
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        # 첫 블록은 다운샘플링 적용 가능성 있음
        self.in_planes = planes # 이후 블록을 위한 입력 채널 업데이트

        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
            # 나머지 블록은 stride=1로 동일한 해상도 유지

        return nn.Sequential(*layers)   # 블록들을 Seguential로 묶어 반환

    def forward(self, x):
        x = self.conv1(x)  # 입력: [B, 1, 224, 224] -> [B, 64, 112, 112]
        x = self.bn1(x)    # 정규화
        x = self.relu(x)   # ReLU 활성화
        x = self.maxpool(x)  # [B, 64, 112, 112] -> [B, 64, 56, 56]

        x = self.layer1(x)  # [B, 64, 56, 56]
        x = self.layer2(x)  # [B, 128, 28, 28]
        x = self.layer3(x)  # [B, 256, 14, 14]
        x = self.layer4(x)  # [B, 512, 7, 7]

        x = self.avgpool(x)  # [B, 512, 1, 1]
        x = torch.flatten(x, 1)  # [B, 512]
        x = self.fc(x)  # [B, num_classes]

        return x


# -------------------- 학습 루프 --------------------
def run(seed=42):
    seed_everything(seed)


    # 모든 CT 슬라이스 파일 경로 불러오기 (LIDC-IDRI 환자 폴더 안의 .npy 파일들)
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))

    # 파일 경로와 해당 파일의 라벨을 튜플로 저장
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    # 라벨이 None이 아닌 데이터만 필터링 (중립 제외)
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]

    # 파일, 라벨을 리스트로 분리
    files, labels = zip(*file_label_pairs)

    # 전체 데이터를 train(70%), val(15%), test(15%)로 분할
    train_files, temp_files, train_labels, temp_labels = train_test_split(
    files, labels, test_size=0.3, random_state=42)

    val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=0.5, random_state=42)

    # 데이터 불러오기
    train_dataset = CTDataset(train_files, train_labels, transform=train_transform)
    val_dataset = CTDataset(val_files, val_labels, transform=val_transform)
    test_dataset = CTDataset(test_files, test_labels, transform=val_transform)

    # 데이터 로더
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    # 모델, 손실함수, 옵티마이저 정의
    model = ResNet18_CBAM().to(device)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.6653, 0.3347], device=device))
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    best_acc = 0.0  # 가장 높은 val accuracy를 저장
    save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "r18_cbam_mga_aug_lr4_ep100_weight_seedfix.pth")

    # 학습 루프 시작
    for epoch in range(num_epochs):
        # MGA 스케쥴링: 초기 lambda -> 점점 증가시킴
        lambda_mga = initial_lambda + (final_lambda - initial_lambda) * (epoch / total_epochs)

        model.train()  # 학습 모드로 변경
        epoch_loss = 0
        correct = 0
        total = 0

        # 한 epoch 동안 모든 train 데이터를 학습
        for images, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images = images.to(device)
            labels = labels.to(device)
            masks = masks.to(device)

            outputs = model(images)  # forward pass
            ce_loss = criterion(outputs, labels)  # cross entropy loss

            # -------------------- MGA Loss 계산 위치 --------------------
            attn_map = model.layer3[1].cbam.last_attention  # attention map 꺼내오기

            if attn_map is not None:
                attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)
                attn_loss = F.mse_loss(attn_map, masks)  # mask와의 MSE loss
                loss = ce_loss + lambda_mga * attn_loss
            else:
                loss = ce_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()

        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")
        print(f"[Epoch {epoch+1}] lambda_mga: {lambda_mga:.4f}")

        torch.cuda.empty_cache(); gc.collect()  # 메모리 정리

        # -------------------- 검증 --------------------
        model.eval()
        correct = 0; total = 0

        with torch.no_grad():
            for iamegs, labels, masks in val_loader:
                iamegs, labels, masks = iamegs.to(device), labels.to(device), masks.to(device)
                outputs = model(iamegs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # -------------------- 테스트 --------------------
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()

    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for images, labels, _ in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    # numpy 배열로 변환
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_probs = np.array(y_probs)

    # 지표 계산
    from sklearn.metrics import (
        classification_report, roc_auc_score, confusion_matrix,
        precision_score, recall_score, balanced_accuracy_score,
        matthews_corrcoef, f1_score
    )

    acc = (y_pred == y_true).mean()
    auc = roc_auc_score(y_true, y_probs)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    specificity = tn / (tn + fp + 1e-6)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    # 📋 출력
    print(f"✅ Test Accuracy         : {acc*100:.2f}%")
    print(f"🎯 AUC                   : {auc:.4f}")
    print(f"📌 Precision             : {precision:.4f}")
    print(f"📌 Recall (Sensitivity)  : {recall:.4f}")
    print(f"📌 Specificity           : {specificity:.4f}")
    print(f"📌 F1 Score              : {f1:.4f}")
    print(f"📌 Balanced Accuracy     : {balanced_acc:.4f}")
    print(f"📌 MCC                   : {mcc:.4f}")
    print("\n📌 Confusion Matrix:")
    print(cm)

    # 📁 CSV로 저장
    test_metrics = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "model": "ResNet18_CBAM_MGA",
        "phase": "test",
        "accuracy": round(acc, 4),
        "auc": round(auc, 4),
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "specificity": round(specificity, 4),
        "f1_score": round(f1, 4),
        "balanced_acc": round(balanced_acc, 4),
        "mcc": round(mcc, 4),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }

    csv_path = "logs/final_test_metrics.csv"
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    file_exists = os.path.exists(csv_path)

    with open(csv_path, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=test_metrics.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(test_metrics)

    print(f"\n📁 테스트 지표 저장 완료: {csv_path}")

# 진입점
if __name__ == "__main__":
    run()


Using device: cuda


[Epoch 1]: 100%|██████████| 234/234 [00:09<00:00, 25.26it/s]


Train Acc: 58.3869, Loss: 0.6954
[Epoch 1] lambda_mga: 0.1000
Val Acc: 0.3912
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:09<00:00, 25.76it/s]


Train Acc: 62.0311, Loss: 0.6558
[Epoch 2] lambda_mga: 0.1040
Val Acc: 0.5650
✅ Saved best model!


...

[Epoch 99]: 100%|██████████| 234/234 [00:09<00:00, 25.17it/s]


Train Acc: 97.7760, Loss: 0.0656
[Epoch 99] lambda_mga: 0.4920
Val Acc: 0.8650


[Epoch 100]: 100%|██████████| 234/234 [00:09<00:00, 25.73it/s]


Train Acc: 97.5884, Loss: 0.0733
[Epoch 100] lambda_mga: 0.4960
Val Acc: 0.8912

📊 Test Evaluation:
✅ Test Accuracy         : 89.38%
🎯 AUC                   : 0.9391
📌 Precision             : 0.9104
📌 Recall (Sensitivity)  : 0.9295
📌 Specificity           : 0.8255
📌 F1 Score              : 0.9199
📌 Balanced Accuracy     : 0.8775
📌 MCC                   : 0.7626

📌 Confusion Matrix:
[[227  48]
 [ 37 488]]

📁 테스트 지표 저장 완료: logs/final_test_metrics.csv


: 

In [2]:
# ✅ 전체코드: ResNet18 + CBAM + MGA + Lambda Scheduling + FocalLoss + TTA
# ✅ 핵심: TTADataset 적용 + 다양한 TTA inference 지원

import os, re, numpy as np, torch, gc, csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from glob import glob
from tqdm import tqdm
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from datetime import datetime
import random

# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 경로 및 하이퍼파라미터
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"
batch_size = 16
num_epochs = 150
learning_rate = 1e-4
initial_lambda = 0.1
final_lambda = 0.5

# FocalLoss
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0):
        super().__init__()
        self.weight = weight
        self.gamma = gamma

    def forward(self, input, target):
        logp = F.log_softmax(input, dim=1)
        ce_loss = F.nll_loss(logp, target, weight=self.weight, reduction='none')
        p = torch.exp(-ce_loss)
        return ((1 - p) ** self.gamma * ce_loss).mean()

# Transform 설정
def get_tta_transforms():
    return [
        A.Compose([A.Resize(224, 224), A.Normalize((0.5,), (0.5,)), ToTensorV2()]),
        A.Compose([A.HorizontalFlip(p=1.0), A.Resize(224, 224), A.Normalize((0.5,), (0.5,)), ToTensorV2()]),
        A.Compose([A.Rotate(limit=15, p=1.0), A.Resize(224, 224), A.Normalize((0.5,), (0.5,)), ToTensorV2()]),
        A.Compose([A.RandomBrightnessContrast(p=1.0), A.Resize(224, 224), A.Normalize((0.5,), (0.5,)), ToTensorV2()])
    ]

tta_transforms = get_tta_transforms()

train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.CoarseDropout(p=0.4, max_holes=1, max_height=32, max_width=32),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])
val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

# 시드고정
def seed_everything(seed=42):
    random.seed(seed)                         # 파이썬 random
    np.random.seed(seed)                      # numpy
    torch.manual_seed(seed)                   # torch CPU
    torch.cuda.manual_seed(seed)              # torch GPU
    torch.cuda.manual_seed_all(seed)          # multi-GPU
    torch.backends.cudnn.deterministic = True # 연산 동일하게
    torch.backends.cudnn.benchmark = False    # 연산 속도 최적화 OFF (같은 연산 보장)

# Bounding Box를 Binary Mask로
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    masks = []
    for bbox in bbox_list:
        mask = np.zeros(image_size, dtype=np.float32)
        x_min, y_min, x_max, y_max = bbox
        mask[y_min:y_max, x_min:x_max] = 1.0
        masks.append(mask)
    masks = np.stack(masks)
    masks = np.expand_dims(masks, axis=1)
    return torch.tensor(masks, dtype=torch.float32)

def load_bbox_dict(csv_path):
    df = pd.read_csv(csv_path)
    bbox_dict = {}
    for _, row in df.iterrows():
        pid = row['pid']
        slice_str = row['slice']
        slice_idx = int(re.findall(r'\d+', str(slice_str))[0])
        fname = f"{pid}_slice{slice_idx:04d}.npy"
        bbox = eval(row['bb'])
        bbox_dict.setdefault(fname, []).append(bbox)
    return bbox_dict

bbox_dict = load_bbox_dict(bbox_csv_path)

def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except:
        return None

# Dataset
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        f = self.paths[idx]
        label = self.labels[idx]
        fname = os.path.basename(f)

        img = np.load(f)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = np.expand_dims(img.astype(np.float32), axis=-1)

        h, w = img.shape[:2]
        if fname in bbox_dict:
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(h, w))[0].numpy()
        else:
            mask = np.zeros((h, w), dtype=np.float32)
        mask = np.expand_dims(mask, axis=-1).astype(np.float32)

        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img = augmented["image"]
            mask = augmented["mask"]
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
            mask = torch.tensor(mask.squeeze(), dtype=torch.float32)

        return img, torch.tensor(label).long(), mask

    def __len__(self):
        return len(self.paths)
        

# TTA용 Dataset
class TTADataset(Dataset):
    def __init__(self, paths, labels, tta_transforms):
        self.paths = paths
        self.labels = labels
        self.tta_transforms = tta_transforms

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        f = self.paths[idx]
        label = self.labels[idx]
        img = np.load(f)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = np.expand_dims(img.astype(np.float32), axis=-1)

        images = [t(image=img)["image"] for t in self.tta_transforms]
        return torch.stack(images), torch.tensor(label).long()


# CBAM + ResNet18 정의
# -------------------- CBAM 정의 (MGA 포함) --------------------
# 2 Step : Channel Attention(어떤 채널에 집중할지) * Spatial Attention(어디에 집중할지) = 최종 Attention

class ChannelAttention(nn.Module):  # 입력 feature map의 채널별 중요도를 계산해서 강조함
    def __init__(self, planes, ratio=16):
        # planes : 입력 채널 수
        # ratio : 중간 채널 축소 비율. 기본 1/16으로 bottlenck 구성
        super().__init__()

        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        # MLP 역할을 하는 1x1 conv 블록 -> 채널 압축 -> 비선형 -> 복원 (shared는 avg/max 둘다에서 같이 씀)

        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()
        # 평균 풀링 / 최대 풀링으로 두가지 전역 정보를 추출
        # 마지막 sigmoid는 attention weight로 스케일링

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))
    # avg & max 풀링 경과를 각각 shape MLP에 통과시키고, 더한 후 sigmoid
    # -> shape : [B, C, 1, 1]
    # -> 채널마다 중요도 weight를 곱하게 됨

class SpatialAttention(nn.Module):  # 공간적으로 어디에 집중할지를 결정 -> 각 채널 내부에서 중요한 위치 찾기

    def __init__(self, k=7):    
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    # 채널 차원은 평균, 최대 두 개만 써서 concat
    # 그걸 1채널로 줄여주는 conv
    # 커널 크기 k=7이면 넓은 영역까지 감지 가능

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))
    # 입력 feature map에서 :
    # 평균, 최대값을 각 spatial 위치별로 구함 -> [B, 1, H, W] 두 개
    # concat -> [B, 2, H, w]
    # conv + sigmoid -> 위치별 중요도 map

class CBAM(nn.Module):  
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()
        self.last_attention = None
    # ChannelAttention, SpatialAttention을 내부에 선언
    # MGA를 위해 마지막 attention map을 저장하는 변수 포함

    def forward(self, x):
        ca_out = self.ca(x) * x
        sa_out = self.sa(ca_out)
        self.last_attention = sa_out
        return sa_out * ca_out
    # 채널 중요도 -> 곱함
    # 위치 중요도 -> 곱함
    # 둘 다 반영된 최종 feature map 리턴



# 학습

def run():
    seed_everything(42)
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)

    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)
    
    test_dataset = TTADataset(test_files, test_labels, tta_transforms)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = ResNet18_CBAM().to(device)
    criterion = FocalLoss(weight=torch.tensor([0.65, 0.35], device=device), gamma=2.0)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_acc = 0.0
    save_path = os.path.join("pth", "r18_cbam_mga_aug_tta.pth")
    os.makedirs("logs", exist_ok=True)
    

    monitor_start = 80
    monitor_window = 10
    monitor_threshold = 0.90
    recent_val_accs = []

    for epoch in range(num_epochs):
        lambda_mga = initial_lambda + (final_lambda - initial_lambda) * (epoch / total_epochs)
        model.train()
        total_loss, correct, total = 0, 0, 0
        for imgs, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            imgs, labels, masks = imgs.to(device), labels.to(device), masks.to(device)
            outputs = model(imgs)
            ce_loss = criterion(outputs, labels)
            attn_map = model.layer3[1].cbam.last_attention  # [B, 1, H, W]
            attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)  # [B, 224, 224]
            masks = masks.view(masks.size(0), 224, 224)  # 강제로 [B, 224, 224]로 reshape
            mga_loss = F.mse_loss(attn_map, masks.float())
            loss = ce_loss + lambda_mga * mga_loss
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        print(f"Train Acc: {correct/total:.4f}, Loss: {total_loss/len(train_loader):.4f}")
        model.eval(); val_correct, val_total = 0, 0
        min_val_acc = 0.85  # 기본 기준선
        patience = 10       # 기다릴 수 있는 횟수
        epochs_no_improve = 0
        with torch.no_grad():
            for imgs, labels, _ in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_total += labels.size(0)
        val_acc = val_correct / val_total
        print(f"Val Acc: {val_acc:.4f}")
        
        recent_val_accs.append(val_acc)
        if len(recent_val_accs) > monitor_window:
            recent_val_accs.pop(0)

        if epoch + 1 >= monitor_start:
            if len(recent_val_accs) == monitor_window and all(acc < monitor_threshold for acc in recent_val_accs):
                print(f"🛑 Early stopping: Epoch {epoch+1} 기준 최근 {monitor_window}번 val_acc < {monitor_threshold}")
                break

    # ---------------- 테스트 ----------------
    model.load_state_dict(torch.load(save_path))
    model.eval()
    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for imgs_batch, labels in test_loader:  # imgs_batch: [B, T, C, H, W]
            labels = labels.to(device)
            B, T, C, H, W = imgs_batch.shape
            imgs_batch = imgs_batch.to(device)  # (B, T, C, H, W)
            imgs_batch = imgs_batch.view(-1, C, H, W)  # (B*T, C, H, W)

            outputs = model(imgs_batch)  # (B*T, num_classes)
            outputs = F.softmax(outputs, dim=1)
            outputs = outputs.view(B, T, -1).mean(dim=1)  # (B, num_classes)

            probs = outputs[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    y_true = np.array(y_true); y_pred = np.array(y_pred); y_probs = np.array(y_probs)
    acc = (y_pred == y_true).mean()
    auc = roc_auc_score(y_true, y_probs)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp + 1e-6)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    print(f"\n📊 Test Evaluation:\n✅ Accuracy: {acc*100:.2f}% | AUC: {auc:.4f} | F1: {f1:.4f}")
    print("Confusion Matrix:", cm)

    from collections import Counter

    counter = Counter(labels)
    print(f"Class distribution: {counter}")

    metrics = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "model": "ResNet18_CBAM_MGA",
        "accuracy": round(acc, 4),
        "auc": round(auc, 4),
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "specificity": round(specificity, 4),
        "f1_score": round(f1, 4),
        "balanced_acc": round(balanced_acc, 4),
        "mcc": round(mcc, 4),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }
    with open("logs/final_test_metrics.csv", 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=metrics.keys())
        if not os.path.exists("logs/final_test_metrics.csv"):
            writer.writeheader()
        writer.writerow(metrics)

# 실행
if __name__ == "__main__":
    run()


Using device: cuda:1


/tmp/ipykernel_2569359/2177874015.py:60: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(p=0.4, max_holes=1, max_height=32, max_width=32),


NameError: name 'ResNet18_CBAM' is not defined

In [ ]:
# r18_cbam_mga_aug_focal150_2.pth
# ✅ 전체코드: ResNet18 + CBAM + MGA + Lambda Scheduling
# ✅ 추천 실험 조합 포함:
# - Weighted FocalLoss (gamma=2.0)
# - Data Aug: Resize + Flip + Rotate(15) + BrightnessContrast + Dropout
# - Evaluation Metrics + CSV 저장 + tqdm

import os, re, numpy as np, torch, gc, csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from datetime import datetime
import random

# 디바이스
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 경로 & 하이퍼파라미터
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"
batch_size = 16
num_epochs = 150
learning_rate = 1e-4
initial_lambda = 0.1
final_lambda = 0.5

# FocalLoss 구현
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0):
        super().__init__()
        self.weight = weight
        self.gamma = gamma

    def forward(self, input, target):
        logp = F.log_softmax(input, dim=1)
        ce_loss = F.nll_loss(logp, target, weight=self.weight, reduction='none')
        p = torch.exp(-ce_loss)
        return ((1 - p) ** self.gamma * ce_loss).mean()

# Transform (Albumentations)
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.CoarseDropout(p=0.4, max_holes=1, max_height=32, max_width=32),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])
val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

# BBox -> 마스크
# -------------------- Bounding Box를 Binary Mask로 --------------------
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    # bbox_list : 한 이미지에 들어있는 bounding box 리스트
    # image_size : 출력할 마스크 크기. 보통 이미지와 동일한 (height, width) -> 디폴트는 224x224
    # bbox들을 binary mask로 바꿔주는 함수
    masks = []  # 여러 개의 bbox가 들어오니까, 각각의 마스크를 하나씩 리스트에 쌓기 위한 빈 리스트
    for bbox in bbox_list:  # bbox_list를 하나씩 돌면서 처리 -> [x_min, y_min, x_max, y_max]네 좌표로 구성된 하나의 사각형 영역 
        mask = np.zeros(image_size, dtype=np.float32)   # 224x224짜리 0으로 꽉 찬 2D 배열을 하나 생성
        # 배경이 흰 종이를 만드는 느낌으로 만들고, 사각 영역만 1로 덧칠할거임
        x_min, y_min, x_max, y_max = bbox   # 각 bbox의 네 좌표값을 각각 변수로 언팩. -> 마스크의 해당 영역에 사각형을 칠하기 위해서
        mask[y_min:y_max, x_min:x_max] = 1.0    # y_min, y_max, x_min, x_max까지의 범위에 1.0을 채워 넣음
        # -> 마스크에서 bbox에 해당하는 사각형 영역만 1(foreground)로 표시됨. 나머진 여전히 0(background)
        masks.append(mask)  # 지금 만든 마스크(2D 배열)를 리스트에 추가 -> [mask1, mask2, ...]이렇게 쌓임

    masks = np.stack(masks) # 리스트를 하나의 3D 배열로 합침 -> shape : [N, H, W] -> N은 bbox 개수
    masks = np.expand_dims(masks, axis=1)   # 텐서 shape을 [N, 1, H, W]로 바꿈
    # PyTorch 모델에서 기대하는 (batch x channel x height x width) 포맷 맞추기

    return torch.tensor(masks, dtype=torch.float32)
    # numpy 배열을 PyTorch 텐서로 변환해서 리턴

    # 한 bbox → 하나의 마스크 → 여러 개면 쌓아서 batch 형태로
# -------------------- Bounding Box CSV 로드 --------------------
def load_bbox_dict(csv_path):
    # csv_path : bounding box 정보가 들어있는 CSV 파일 경로
    # 반환값 : {filename:[bbox1, bbox2, ...]} 형태의 딕셔너리
    df = pd.read_csv(csv_path)  # CSV파일을 pandas DataFrame으로 읽어옴
    bbox_dict = {}
    # key : 슬라이스 파일 이름 (ex. "LIDC-IDRI-1012_slice0004.npy")
    # value : 해당 슬라이스에 존재하는 bbox들의 리스트
    for _, row in df.iterrows():    # DataFrame의 모든 행(row)를 하나씩 순회
        # row는 한 줄(=한 bbox)의 정보를 담고 있음

        pid = row['pid']    # 환자 ID (예: "LIDC-IDRI-1012") -> 이미지 이름 구성 요소
        slice_str = row['slice']    # 슬라이스 정보가 들어있는 문자열 (예: "slice_0039")
        slice_idx = int(re.findall(r'\d+', str(slice_str))[0])  # re.findall()로 문자열에서 숫자만 뽑아냄
        # "slice_0039" -> ['0039'] -> [0] -> 39 (슬라이스 번호를 정수로 추출함)
        fname = f"{pid}_slice{slice_idx:04d}.npy"   # 파일명 구성 (예: "LIDC-IDRI-1012_slice0039.npy")
        # {:04d}는 4자리 정수로 만들고 빈자리는 0으로 채워줌 (39 -> 0039)
        bbox = eval(row['bb'])  # row['bb']는 문자열 형태의 bbox (예: "[20, 30, 80, 100]")
        # eval()을 써서 문자열을 리스트로 바꿔줌
        # 주의 : 보안 상 위험할 수 있는 함수지만, 여긴 내부 데이터라 사용중
        bbox_dict.setdefault(fname, []).append(bbox)    # fname이라는 key가 딕셔너리에 없으면 []로 초기화하고,
        # 거기에 bbox를 append -> 슬라이스 하나에 bbox 여러개 있어도 전부 리스트로 모아줌
    return bbox_dict    # 최종적으로 {filename: [bbox1, bbox2, ...]} 형태의 딕셔너리 반환

bbox_dict = load_bbox_dict(bbox_csv_path)
# 실제로 csv_path에 있는 정보를 불러와서 bbox_dict에 저장함
# 이걸 나주에 Dataset 클래스에서 fname 기준을 꺼내쓰게 됨

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname): # fname : 파일 이름 (예: "LIDC-IDRI-1012_slice0039_5.npy")
    # 이 이름에서 malignancy score(악성도 점수)를 추출해서 라벨로 변환
    try:    # 파일명이 이상하거나 에러나면 except로 빠져나가서 None 반환함 (안전장치)
        score = int(fname.split("_")[-1].replace(".npy", ""))
        # 파일명에서 _ 제외하고 나머지 것들 중에 마지막에꺼를 가져와서 .npy를 "" 이렇게 공백으로 처리함
        # fname.split("_") -> ['LIDC-IDRI-1012', 'slice0039', '5.npy]
        # [-1] -> '5.npy'
        # .replace(".npy", "") -> '5'
        # int(...) -> 5 <- 이게 malignancy score
        return None if score == 3 else int(score >= 4)
        # 라벨 결정 로직으로
        # score == 3 -> 중립 -> None 반환 -> 학습에서 제외
        # score >= 4 -> 암(양성) -> 1
        # score <= 2 -> 정상(음성) -> 0
        # int(score >= 4)는 파이썬에서 True -> 1
        # False -> 0 이니깐 자동으로 라벨이 됨
    except:
        return None
        # 혹시 split이나 replace, int 변환이 실패하면 그냥 None 반환하고 무시

# 시드 고정
def seed_everything(seed=42):
    random.seed(seed)                         # 파이썬 random
    np.random.seed(seed)                      # numpy
    torch.manual_seed(seed)                   # torch CPU
    torch.cuda.manual_seed(seed)              # torch GPU
    torch.cuda.manual_seed_all(seed)          # multi-GPU
    torch.backends.cudnn.deterministic = True # 연산 동일하게
    torch.backends.cudnn.benchmark = False    # 연산 속도 최적화 OFF (같은 연산 보장)

    
# Dataset (Albumentations 적용)
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        f = self.paths[idx]
        label = self.labels[idx]
        fname = os.path.basename(f)

        img = np.load(f)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        mask = np.zeros((224, 224), dtype=np.float32)
        if fname in bbox_dict:
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(224, 224)).sum(0).squeeze().numpy()

        augmented = self.transform(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask']
        return img.unsqueeze(0), torch.tensor(label).long(), mask

    def __len__(self): return len(self.paths)

# CBAM + ResNet18 정의
...

# 학습

def run():
    seed_everything(42)
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)

    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size)

    model = ResNet18_CBAM().to(device)
    criterion = FocalLoss(weight=torch.tensor([0.65, 0.35], device=device), gamma=2.0)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_acc = 0.0
    save_path = os.path.join("pth", "r18_cbam_mga_aug_focal150_2.pth")
    os.makedirs("logs", exist_ok=True)

    for epoch in range(num_epochs):
        lambda_mga = initial_lambda + (final_lambda - initial_lambda) * (epoch / total_epochs)
        model.train()
        total_loss, correct, total = 0, 0, 0
        for imgs, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            imgs, labels, masks = imgs.to(device), labels.to(device), masks.to(device)
            outputs = model(imgs)
            ce_loss = criterion(outputs, labels)
            attn_map = model.layer3[1].cbam.last_attention
            attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)
            mga_loss = F.mse_loss(attn_map, masks)
            loss = ce_loss + lambda_mga * mga_loss
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        print(f"Train Acc: {correct/total:.4f}, Loss: {total_loss/len(train_loader):.4f}")
        model.eval(); val_correct, val_total = 0, 0
        with torch.no_grad():
            for imgs, labels, _ in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_total += labels.size(0)
        val_acc = val_correct / val_total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # ---------------- 테스트 ----------------
    model.load_state_dict(torch.load(save_path))
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels, _ in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_probs.extend(probs.cpu().numpy())

    y_true = np.array(y_true); y_pred = np.array(y_pred); y_probs = np.array(y_probs)
    acc = (y_pred == y_true).mean()
    auc = roc_auc_score(y_true, y_probs)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp + 1e-6)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    print(f"\n📊 Test Evaluation:\n✅ Accuracy: {acc*100:.2f}% | AUC: {auc:.4f} | F1: {f1:.4f}")
    print("Confusion Matrix:", cm)

    metrics = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "model": "ResNet18_CBAM_MGA",
        "accuracy": round(acc, 4),
        "auc": round(auc, 4),
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "specificity": round(specificity, 4),
        "f1_score": round(f1, 4),
        "balanced_acc": round(balanced_acc, 4),
        "mcc": round(mcc, 4),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }
    with open("logs/final_test_metrics.csv", 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=metrics.keys())
        if not os.path.exists("logs/final_test_metrics.csv"):
            writer.writeheader()
        writer.writerow(metrics)

# 실행
if __name__ == "__main__":
    run()


In [ ]:
# ✅ Test Accuracy: 88.50% | AUC: 0.9253 | F1: 0.9148
# Confusion Matrix: [[214  61]
#  [ 31 494]]
#r18_cbam_mga_aug_focal150.pth

# ✅ 전체코드: ResNet18 + CBAM + MGA + Lambda Scheduling

# ✅ 추천 실험 조합 포함:
# - Weighted FocalLoss (gamma=2.0)
# - Data Aug: Resize + Flip + Rotate(15) + BrightnessContrast + Dropout
# - Evaluation Metrics + CSV 저장 + tqdm

import os, re, numpy as np, torch, gc, csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from datetime import datetime

# 디바이스
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 경로 & 하이퍼파라미터
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"
batch_size = 16
num_epochs = 150
learning_rate = 1e-4
initial_lambda = 0.1
final_lambda = 0.5

# FocalLoss 구현
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0):
        super().__init__()
        self.weight = weight
        self.gamma = gamma

    def forward(self, input, target):
        logp = F.log_softmax(input, dim=1)
        ce_loss = F.nll_loss(logp, target, weight=self.weight, reduction='none')
        p = torch.exp(-ce_loss)
        return ((1 - p) ** self.gamma * ce_loss).mean()

# Transform (Albumentations)
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.CoarseDropout(p=0.4, max_holes=1, max_height=32, max_width=32),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])
val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

# -------------------- Bounding Box를 Binary Mask로 --------------------
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    # bbox_list : 한 이미지에 들어있는 bounding box 리스트
    # image_size : 출력할 마스크 크기. 보통 이미지와 동일한 (height, width) -> 디폴트는 224x224
    # bbox들을 binary mask로 바꿔주는 함수
    masks = []  # 여러 개의 bbox가 들어오니까, 각각의 마스크를 하나씩 리스트에 쌓기 위한 빈 리스트
    for bbox in bbox_list:  # bbox_list를 하나씩 돌면서 처리 -> [x_min, y_min, x_max, y_max]네 좌표로 구성된 하나의 사각형 영역 
        mask = np.zeros(image_size, dtype=np.float32)   # 224x224짜리 0으로 꽉 찬 2D 배열을 하나 생성
        # 배경이 흰 종이를 만드는 느낌으로 만들고, 사각 영역만 1로 덧칠할거임
        x_min, y_min, x_max, y_max = bbox   # 각 bbox의 네 좌표값을 각각 변수로 언팩. -> 마스크의 해당 영역에 사각형을 칠하기 위해서
        mask[y_min:y_max, x_min:x_max] = 1.0    # y_min, y_max, x_min, x_max까지의 범위에 1.0을 채워 넣음
        # -> 마스크에서 bbox에 해당하는 사각형 영역만 1(foreground)로 표시됨. 나머진 여전히 0(background)
        masks.append(mask)  # 지금 만든 마스크(2D 배열)를 리스트에 추가 -> [mask1, mask2, ...]이렇게 쌓임

    masks = np.stack(masks) # 리스트를 하나의 3D 배열로 합침 -> shape : [N, H, W] -> N은 bbox 개수
    masks = np.expand_dims(masks, axis=1)   # 텐서 shape을 [N, 1, H, W]로 바꿈
    # PyTorch 모델에서 기대하는 (batch x channel x height x width) 포맷 맞추기

    return torch.tensor(masks, dtype=torch.float32)
    # numpy 배열을 PyTorch 텐서로 변환해서 리턴

    # 한 bbox → 하나의 마스크 → 여러 개면 쌓아서 batch 형태로
# -------------------- Bounding Box CSV 로드 --------------------
def load_bbox_dict(csv_path):
    # csv_path : bounding box 정보가 들어있는 CSV 파일 경로
    # 반환값 : {filename:[bbox1, bbox2, ...]} 형태의 딕셔너리
    df = pd.read_csv(csv_path)  # CSV파일을 pandas DataFrame으로 읽어옴
    bbox_dict = {}
    # key : 슬라이스 파일 이름 (ex. "LIDC-IDRI-1012_slice0004.npy")
    # value : 해당 슬라이스에 존재하는 bbox들의 리스트
    for _, row in df.iterrows():    # DataFrame의 모든 행(row)를 하나씩 순회
        # row는 한 줄(=한 bbox)의 정보를 담고 있음

        pid = row['pid']    # 환자 ID (예: "LIDC-IDRI-1012") -> 이미지 이름 구성 요소
        slice_str = row['slice']    # 슬라이스 정보가 들어있는 문자열 (예: "slice_0039")
        slice_idx = int(re.findall(r'\d+', str(slice_str))[0])  # re.findall()로 문자열에서 숫자만 뽑아냄
        # "slice_0039" -> ['0039'] -> [0] -> 39 (슬라이스 번호를 정수로 추출함)
        fname = f"{pid}_slice{slice_idx:04d}.npy"   # 파일명 구성 (예: "LIDC-IDRI-1012_slice0039.npy")
        # {:04d}는 4자리 정수로 만들고 빈자리는 0으로 채워줌 (39 -> 0039)
        bbox = eval(row['bb'])  # row['bb']는 문자열 형태의 bbox (예: "[20, 30, 80, 100]")
        # eval()을 써서 문자열을 리스트로 바꿔줌
        # 주의 : 보안 상 위험할 수 있는 함수지만, 여긴 내부 데이터라 사용중
        bbox_dict.setdefault(fname, []).append(bbox)    # fname이라는 key가 딕셔너리에 없으면 []로 초기화하고,
        # 거기에 bbox를 append -> 슬라이스 하나에 bbox 여러개 있어도 전부 리스트로 모아줌
    return bbox_dict    # 최종적으로 {filename: [bbox1, bbox2, ...]} 형태의 딕셔너리 반환

bbox_dict = load_bbox_dict(bbox_csv_path)
# 실제로 csv_path에 있는 정보를 불러와서 bbox_dict에 저장함
# 이걸 나주에 Dataset 클래스에서 fname 기준을 꺼내쓰게 됨

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname): # fname : 파일 이름 (예: "LIDC-IDRI-1012_slice0039_5.npy")
    # 이 이름에서 malignancy score(악성도 점수)를 추출해서 라벨로 변환
    try:    # 파일명이 이상하거나 에러나면 except로 빠져나가서 None 반환함 (안전장치)
        score = int(fname.split("_")[-1].replace(".npy", ""))
        # 파일명에서 _ 제외하고 나머지 것들 중에 마지막에꺼를 가져와서 .npy를 "" 이렇게 공백으로 처리함
        # fname.split("_") -> ['LIDC-IDRI-1012', 'slice0039', '5.npy]
        # [-1] -> '5.npy'
        # .replace(".npy", "") -> '5'
        # int(...) -> 5 <- 이게 malignancy score
        return None if score == 3 else int(score >= 4)
        # 라벨 결정 로직으로
        # score == 3 -> 중립 -> None 반환 -> 학습에서 제외
        # score >= 4 -> 암(양성) -> 1
        # score <= 2 -> 정상(음성) -> 0
        # int(score >= 4)는 파이썬에서 True -> 1
        # False -> 0 이니깐 자동으로 라벨이 됨
    except:
        return None
        # 혹시 split이나 replace, int 변환이 실패하면 그냥 None 반환하고 무시

# Dataset (Albumentations 적용)
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        f = self.paths[idx]
        label = self.labels[idx]
        fname = os.path.basename(f)

        img = np.load(f)  # 원래 이미지
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = img.astype(np.float32)
        img = np.expand_dims(img, axis=-1)  # (H, W, 1)

        h, w = img.shape[:2]

        # === (1) 원본 이미지와 같은 크기로 마스크 생성 ===
        if fname in bbox_dict:
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(h, w))[0].numpy()
        else:
            mask = np.zeros((h, w), dtype=np.float32)

        mask = np.expand_dims(mask, axis=-1).astype(np.float32)  # (H, W, 1)

        # === (2) Albumentations transform: 같이 resize 시킴 ===
        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img = augmented["image"]
            mask = augmented["mask"]
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
            mask = torch.tensor(mask.squeeze(), dtype=torch.float32)

        return img, torch.tensor(label).long(), mask

    def __len__(self):
        return len(self.paths)

# CBAM + ResNet18 정의
# -------------------- CBAM 정의 (MGA 포함) --------------------
# 2 Step : Channel Attention(어떤 채널에 집중할지) * Spatial Attention(어디에 집중할지) = 최종 Attention

class ChannelAttention(nn.Module):  # 입력 feature map의 채널별 중요도를 계산해서 강조함
    def __init__(self, planes, ratio=16):
        # planes : 입력 채널 수
        # ratio : 중간 채널 축소 비율. 기본 1/16으로 bottlenck 구성
        super().__init__()

        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        # MLP 역할을 하는 1x1 conv 블록 -> 채널 압축 -> 비선형 -> 복원 (shared는 avg/max 둘다에서 같이 씀)

        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()
        # 평균 풀링 / 최대 풀링으로 두가지 전역 정보를 추출
        # 마지막 sigmoid는 attention weight로 스케일링

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))
    # avg & max 풀링 경과를 각각 shape MLP에 통과시키고, 더한 후 sigmoid
    # -> shape : [B, C, 1, 1]
    # -> 채널마다 중요도 weight를 곱하게 됨

class SpatialAttention(nn.Module):  # 공간적으로 어디에 집중할지를 결정 -> 각 채널 내부에서 중요한 위치 찾기

    def __init__(self, k=7):    
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    # 채널 차원은 평균, 최대 두 개만 써서 concat
    # 그걸 1채널로 줄여주는 conv
    # 커널 크기 k=7이면 넓은 영역까지 감지 가능

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))
    # 입력 feature map에서 :
    # 평균, 최대값을 각 spatial 위치별로 구함 -> [B, 1, H, W] 두 개
    # concat -> [B, 2, H, w]
    # conv + sigmoid -> 위치별 중요도 map

class CBAM(nn.Module):  
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()
        self.last_attention = None
    # ChannelAttention, SpatialAttention을 내부에 선언
    # MGA를 위해 마지막 attention map을 저장하는 변수 포함

    def forward(self, x):
        ca_out = self.ca(x) * x
        sa_out = self.sa(ca_out)
        self.last_attention = sa_out
        return sa_out * ca_out
    # 채널 중요도 -> 곱함
    # 위치 중요도 -> 곱함
    # 둘 다 반영된 최종 feature map 리턴



# 학습

def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)

    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size)

    model = ResNet18_CBAM().to(device)
    criterion = FocalLoss(weight=torch.tensor([0.65, 0.35], device=device), gamma=2.0)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_acc = 0.0
    save_path = os.path.join("pth", "r18_cbam_mga_aug_focal150.pth")
    os.makedirs("logs", exist_ok=True)

    for epoch in range(num_epochs):
        lambda_mga = initial_lambda + (final_lambda - initial_lambda) * (epoch / total_epochs)
        model.train()
        total_loss, correct, total = 0, 0, 0
        for imgs, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            imgs, labels, masks = imgs.to(device), labels.to(device), masks.to(device)
            outputs = model(imgs)
            ce_loss = criterion(outputs, labels)
            attn_map = model.layer3[1].cbam.last_attention  # [B, 1, H, W]
            attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)  # [B, 224, 224]
            masks = masks.view(masks.size(0), 224, 224)  # 강제로 [B, 224, 224]로 reshape
            mga_loss = F.mse_loss(attn_map, masks.float())
            loss = ce_loss + lambda_mga * mga_loss
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        print(f"Train Acc: {correct/total:.4f}, Loss: {total_loss/len(train_loader):.4f}")
        model.eval(); val_correct, val_total = 0, 0
        with torch.no_grad():
            for imgs, labels, _ in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_total += labels.size(0)
        val_acc = val_correct / val_total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # ---------------- 테스트 ----------------
    model.load_state_dict(torch.load(save_path))
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels, _ in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_probs.extend(probs.cpu().numpy())

    y_true = np.array(y_true); y_pred = np.array(y_pred); y_probs = np.array(y_probs)
    acc = (y_pred == y_true).mean()
    auc = roc_auc_score(y_true, y_probs)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp + 1e-6)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    print(f"\n📊 Test Evaluation:\n✅ Accuracy: {acc*100:.2f}% | AUC: {auc:.4f} | F1: {f1:.4f}")
    print("Confusion Matrix:", cm)

    metrics = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "model": "ResNet18_CBAM_MGA",
        "accuracy": round(acc, 4),
        "auc": round(auc, 4),
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "specificity": round(specificity, 4),
        "f1_score": round(f1, 4),
        "balanced_acc": round(balanced_acc, 4),
        "mcc": round(mcc, 4),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }
    with open("logs/final_test_metrics.csv", 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=metrics.keys())
        if not os.path.exists("logs/final_test_metrics.csv"):
            writer.writeheader()
        writer.writerow(metrics)

# 실행
if __name__ == "__main__":
    run()


Using device: cuda:1


/tmp/ipykernel_1974167/4093859747.py:52: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(p=0.4, max_holes=1, max_height=32, max_width=32),
[Epoch 1]: 100%|██████████| 234/234 [00:13<00:00, 17.91it/s]


Train Acc: 0.3540, Loss: 0.0447
Val Acc: 0.3113
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:12<00:00, 18.60it/s]


Train Acc: 0.3631, Loss: 0.0262
Val Acc: 0.3113


...

Train Acc: 0.9705, Loss: 0.0029
Val Acc: 0.8475


[Epoch 149]: 100%|██████████| 234/234 [00:13<00:00, 17.87it/s]


Train Acc: 0.9703, Loss: 0.0027
Val Acc: 0.7275


[Epoch 150]: 100%|██████████| 234/234 [00:13<00:00, 17.49it/s]


Train Acc: 0.9665, Loss: 0.0035
Val Acc: 0.6362

📊 Test Evaluation:
✅ Accuracy: 88.50% | AUC: 0.9253 | F1: 0.9148
Confusion Matrix: [[214  61]
 [ 31 494]]


In [ ]:
# ✅ Test Accuracy         : 87.12%
# 🎯 AUC                   : 0.9110
# 📌 Precision             : 0.9011
# 📌 Recall (Sensitivity)  : 0.9029
# 📌 Specificity           : 0.8109
# 📌 F1 Score              : 0.9020
# 📌 Balanced Accuracy     : 0.8569
# 📌 MCC                   : 0.7144

# 📌 Confusion Matrix:
# [[223  52]
#  [ 51 474]]
# r18_cbam_mga_aug_lr4_ep100_label1.pth
# 전체코드: ResNet18 + CBAM + MGA Loss + Lambda Scheduling (Label Smoothing + )

import os, re, numpy as np, torch, gc
import csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import pandas as pd
import cv2
import torchvision.transforms as transforms
from PIL import Image
from datetime import datetime
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random

# -------------------- 디바이스 설정 --------------------
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------- 하이퍼파라미터 설정 --------------------
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"

batch_size = 8
num_epochs = 100
learning_rate = 1e-4

# lambda MGA 스케줄 설정
initial_lambda = 0.1
final_lambda = 0.5
total_epochs = num_epochs

# -------------------- Transform --------------------
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.CoarseDropout(
        max_holes=1, max_height=32, max_width=32, min_holes=1, 
        min_height=16, min_width=16, fill_value=0, p=0.5),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        file_path = self.paths[idx]
        label = self.labels[idx]
        fname = os.path.basename(file_path)

        img = np.load(file_path)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = img.astype(np.float32)

        if fname in bbox_dict:
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(img.shape[0], img.shape[1])).sum(axis=0)
        else:
            mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)

        if self.transform:
            transformed = self.transform(image=img, mask=mask)
            img = transformed['image']
            mask = transformed['mask']
        else:
            img = torch.tensor(img).unsqueeze(0)
            mask = torch.tensor(mask)

        return img, torch.tensor(label).long(), mask

    def __len__(self):
        return len(self.paths)

# -------------------- Bounding Box Mask --------------------
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    masks = []
    for bbox in bbox_list:
        mask = np.zeros(image_size, dtype=np.float32)
        x_min, y_min, x_max, y_max = bbox
        mask[y_min:y_max, x_min:x_max] = 1.0
        masks.append(mask)
    masks = np.stack(masks)
    masks = np.expand_dims(masks, axis=1)
    return torch.tensor(masks, dtype=torch.float32)

def seed_everything(seed=42):
    random.seed(seed)                         # 파이썬 random
    np.random.seed(seed)                      # numpy
    torch.manual_seed(seed)                   # torch CPU
    torch.cuda.manual_seed(seed)              # torch GPU
    torch.cuda.manual_seed_all(seed)          # multi-GPU
    torch.backends.cudnn.deterministic = True # 연산 동일하게
    torch.backends.cudnn.benchmark = False    # 연산 속도 최적화 OFF (같은 연산 보장)

# -------------------- Bounding Box CSV --------------------
def load_bbox_dict(csv_path):
    df = pd.read_csv(csv_path)
    bbox_dict = {}
    for _, row in df.iterrows():
        pid = row['pid']
        slice_idx = int(re.findall(r'\d+', str(row['slice']))[0])
        fname = f"{pid}_slice{slice_idx:04d}.npy"
        bbox = eval(row['bb'])
        bbox_dict.setdefault(fname, []).append(bbox)
    return bbox_dict

bbox_dict = load_bbox_dict(bbox_csv_path)

def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except:
        return None

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname): # fname : 파일 이름 (예: "LIDC-IDRI-1012_slice0039_5.npy")
    # 이 이름에서 malignancy score(악성도 점수)를 추출해서 라벨로 변환
    try:    # 파일명이 이상하거나 에러나면 except로 빠져나가서 None 반환함 (안전장치)
        score = int(fname.split("_")[-1].replace(".npy", ""))
        # 파일명에서 _ 제외하고 나머지 것들 중에 마지막에꺼를 가져와서 .npy를 "" 이렇게 공백으로 처리함
        # fname.split("_") -> ['LIDC-IDRI-1012', 'slice0039', '5.npy]
        # [-1] -> '5.npy'
        # .replace(".npy", "") -> '5'
        # int(...) -> 5 <- 이게 malignancy score
        return None if score == 3 else int(score >= 4)
        # 라벨 결정 로직으로
        # score == 3 -> 중립 -> None 반환 -> 학습에서 제외
        # score >= 4 -> 암(양성) -> 1
        # score <= 2 -> 정상(음성) -> 0
        # int(score >= 4)는 파이썬에서 True -> 1
        # False -> 0 이니깐 자동으로 라벨이 됨
    except:
        return None
        # 혹시 split이나 replace, int 변환이 실패하면 그냥 None 반환하고 무시

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    # PyTorch의 Dataset 클래스를 상속해서 커ㅡ텀 데이터셋 정의
    # 나중에 DataLoader랑 같이 쓰이기 때문에 __len__()이랑 __getitem__()을 꼭 넣어줘야함
    def __init__(self, paths, labels, transform=None):  # 생성자 : 세개의 인자를 받음
        # paths : 이미지 .npy 파일 경로 리스트
        # labels : 각 이미지에 대한 라벨 리스트 (0, 1 or None)
        # transform : 이미지 증강 설정 (train_transform, val_transform 등)
        self.paths = paths
        self.labels = labels
        self.transform = transform
        # 받은 인자를 멤버 변수로 저장. 나중에 gettem()에서 접근함

    def __getitem__(self, idx): # DataLoader가 이걸 호출할 때 index에 해당하는 sample 하나를 반환
        # 이미지, 라벨, 마스크( = MGA용 target) 3개를 리턴함
        file_path = self.paths[idx] # 파일 경로 불러오기
        label = self.labels[idx]    # 라벨 불러오기
        fname = os.path.basename(file_path) # 전체 경로에서 파일 이름만 추출 -> 나중에 bbox_dict[fname] 찾을때 쓰임

        img = np.load(file_path)    # .npy 파일에서 CT 슬라이스 불러오기 -> 흑백 CT 이미지, shape은 (H, W)
        img = np.clip(img, -1000, 400)  # CT 이미지 HU 값이 너무 크거나 작으면 노이즈 -> -1000(공기) ~ 400(연조직)으로 클리핑해서 노이즈 제거
        img = (img + 1000) / 1400.  # 정규화 : -1000 -> 0, 400 -> 1 사이 값으로 바꿔줌 -> 모델이 안정적으로 학습할 수 있도록 함
        img = img.astype(np.float32)
        
        h, w = img.shape
        img = np.expand_dims(img, axis=-1)  # (H, W, 1)

        # 마스크 생성
        if fname in bbox_dict:
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(h, w)).sum(dim=0).numpy()
        else:
            mask = np.zeros((h, w), dtype=np.float32)

        # transform 적용
        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img = augmented['image']  # shape: [1, 224, 224]
            mask = augmented['mask']  # shape: [224, 224]
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
            mask = torch.tensor(mask, dtype=torch.float32)

        return img, torch.tensor(label).long(), mask
        
    def __len__(self):
        return len(self.paths)
    # 전체 데이터셋 길이 반환 -> DataLoader가 아라야 배치 쪼갤 수 있음.

# -------------------- CBAM 정의 (MGA 포함) --------------------
# 2 Step : Channel Attention(어떤 채널에 집중할지) * Spatial Attention(어디에 집중할지) = 최종 Attention

class ChannelAttention(nn.Module):  # 입력 feature map의 채널별 중요도를 계산해서 강조함
    def __init__(self, planes, ratio=16):
        # planes : 입력 채널 수
        # ratio : 중간 채널 축소 비율. 기본 1/16으로 bottlenck 구성
        super().__init__()

        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        # MLP 역할을 하는 1x1 conv 블록 -> 채널 압축 -> 비선형 -> 복원 (shared는 avg/max 둘다에서 같이 씀)

        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()
        # 평균 풀링 / 최대 풀링으로 두가지 전역 정보를 추출
        # 마지막 sigmoid는 attention weight로 스케일링

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))
    # avg & max 풀링 경과를 각각 shape MLP에 통과시키고, 더한 후 sigmoid
    # -> shape : [B, C, 1, 1]
    # -> 채널마다 중요도 weight를 곱하게 됨

class SpatialAttention(nn.Module):  # 공간적으로 어디에 집중할지를 결정 -> 각 채널 내부에서 중요한 위치 찾기

    def __init__(self, k=7):    
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    # 채널 차원은 평균, 최대 두 개만 써서 concat
    # 그걸 1채널로 줄여주는 conv
    # 커널 크기 k=7이면 넓은 영역까지 감지 가능

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))
    # 입력 feature map에서 :
    # 평균, 최대값을 각 spatial 위치별로 구함 -> [B, 1, H, W] 두 개
    # concat -> [B, 2, H, w]
    # conv + sigmoid -> 위치별 중요도 map

class CBAM(nn.Module):  
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()
        self.last_attention = None
    # ChannelAttention, SpatialAttention을 내부에 선언
    # MGA를 위해 마지막 attention map을 저장하는 변수 포함

    def forward(self, x):
        ca_out = self.ca(x) * x
        sa_out = self.sa(ca_out)
        self.last_attention = sa_out
        return sa_out * ca_out
    # 채널 중요도 -> 곱함
    # 위치 중요도 -> 곱함
    # 둘 다 반영된 최종 feature map 리턴

# -------------------- ResNet18 + CBAM 모델 정의 --------------------
# BasicBlockCBAM : ResNet의 기본 Residual Block 하나를 정의
# → conv → BN → ReLU → conv → BN → (CBAM optional) → Add → ReLU

# ResNet18_CBAM : ResNet18 구조로 전체 네트워크 쌓기
# → conv1 → layer1~3 → layer4 → avgpool → fc

class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        # 입력 채널: in_planes, 출력 채널: out_planes, 3x3 커널, padding=1로 크기 유지, stride로 크기 조절
        self.bn1 = nn.BatchNorm2d(out_planes)   # 배치 정규화
        self.relu = nn.ReLU()   # 비선형 활성화 함수

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        # 두번째 conv, 채널 수 유지, 크기 유지
        self.bn2 = nn.BatchNorm2d(out_planes)   # 배치 정규화

        self.cbam = CBAM(out_planes) if use_cbam else None  # CBAM 모듈 사용 여부
        self.downsample = downsample    # residual 연결 시 차원 맞추는 conv

    def forward(self, x):
        residual = x    # skip connection용 입력 저장

        out = self.conv1(x) # 첫 번째 conv
        out = self.bn1(out) # 정규화
        out = self.relu(out)  # 활성화

        out = self.conv2(out)   # 두 번째 conv
        out = self.bn2(out) # 정규화

        if self.cbam:
            out = self.cbam(out)    # CBAM 적용

        if self.downsample:
            residual = self.downsample(x)   # shortcut 경로 보정

        out += residual # skip connection
        out = self.relu(out)    # 출력에 ReLU 적용

        return out  # 결과 반환

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2, use_cbam=True):
        super().__init__()
        self.in_planes = 64
        self.use_cbam = use_cbam
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(64, blocks=2, use_cbam=use_cbam)
        self.layer2 = self._make_layer(128, blocks=2, stride=2, use_cbam=use_cbam)
        self.layer3 = self._make_layer(256, blocks=2, stride=2, use_cbam=use_cbam)
        self.layer4 = self._make_layer(512, blocks=2, stride=2, use_cbam=False)  # 마지막 블록은 CBAM 제거
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)


# -------------------- 학습 루프 --------------------
def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)

    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_dataset = CTDataset(train_files, train_labels, transform=train_transform)
    val_dataset = CTDataset(val_files, val_labels, transform=val_transform)
    test_dataset = CTDataset(test_files, test_labels, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # 모델, 손실함수, 옵티마이저 정의
    model = ResNet18_CBAM().to(device)

    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0  # 가장 높은 val accuracy를 저장
    save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "r18_cbam_mga_aug_lr4_ep100_label1.pth")

    # 학습 루프 시작
    for epoch in range(num_epochs):
        # MGA 스케쥴링: 초기 lambda -> 점점 증가시킴
        lambda_mga = initial_lambda + (final_lambda - initial_lambda) * (epoch / total_epochs)

        model.train()  # 학습 모드로 변경
        epoch_loss = 0
        correct = 0
        total = 0

        # 한 epoch 동안 모든 train 데이터를 학습
        for images, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images = images.to(device)
            labels = labels.to(device)
            masks = masks.to(device)

            outputs = model(images)  # forward pass
            ce_loss = criterion(outputs, labels)  # cross entropy loss

            # -------------------- MGA Loss 계산 위치 --------------------
            attn_map = model.layer3[1].cbam.last_attention  # attention map 꺼내오기

            if attn_map is not None:
                attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)
                attn_loss = F.mse_loss(attn_map, masks)  # mask와의 MSE loss
                loss = ce_loss + lambda_mga * attn_loss
            else:
                loss = ce_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()

        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")
        print(f"[Epoch {epoch+1}] lambda_mga: {lambda_mga:.4f}")

        torch.cuda.empty_cache(); gc.collect()  # 메모리 정리

        # -------------------- 검증 --------------------
        model.eval()
        correct = 0; total = 0

        with torch.no_grad():
            for iamegs, labels, masks in val_loader:
                iamegs, labels, masks = iamegs.to(device), labels.to(device), masks.to(device)
                outputs = model(iamegs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # -------------------- 테스트 --------------------
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()

    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for images, labels, _ in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    # numpy 배열로 변환
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_probs = np.array(y_probs)

    # 지표 계산
    from sklearn.metrics import (
        classification_report, roc_auc_score, confusion_matrix,
        precision_score, recall_score, balanced_accuracy_score,
        matthews_corrcoef, f1_score
    )

    acc = (y_pred == y_true).mean()
    auc = roc_auc_score(y_true, y_probs)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    specificity = tn / (tn + fp + 1e-6)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    # 📋 출력
    print(f"✅ Test Accuracy         : {acc*100:.2f}%")
    print(f"🎯 AUC                   : {auc:.4f}")
    print(f"📌 Precision             : {precision:.4f}")
    print(f"📌 Recall (Sensitivity)  : {recall:.4f}")
    print(f"📌 Specificity           : {specificity:.4f}")
    print(f"📌 F1 Score              : {f1:.4f}")
    print(f"📌 Balanced Accuracy     : {balanced_acc:.4f}")
    print(f"📌 MCC                   : {mcc:.4f}")
    print("\n📌 Confusion Matrix:")
    print(cm)

    # 📁 CSV로 저장
    test_metrics = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "model": "ResNet18_CBAM_MGA",
        "phase": "test",
        "accuracy": round(acc, 4),
        "auc": round(auc, 4),
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "specificity": round(specificity, 4),
        "f1_score": round(f1, 4),
        "balanced_acc": round(balanced_acc, 4),
        "mcc": round(mcc, 4),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }

    csv_path = "logs/final_test_metrics.csv"
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    file_exists = os.path.exists(csv_path)

    with open(csv_path, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=test_metrics.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(test_metrics)

    print(f"\n📁 테스트 지표 저장 완료: {csv_path}")

# 진입점
if __name__ == "__main__":
    run()


Using device: cuda:1


/tmp/ipykernel_1974167/24495587.py:42: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(
[Epoch 1]: 100%|██████████| 467/467 [00:15<00:00, 29.86it/s]


Train Acc: 65.2465, Loss: 0.6626
[Epoch 1] lambda_mga: 0.1000
Val Acc: 0.6675
✅ Saved best model!


[Epoch 2]: 100%|██████████| 467/467 [00:15<00:00, 29.76it/s]


Train Acc: 65.1661, Loss: 0.6476
[Epoch 2] lambda_mga: 0.1040
Val Acc: 0.6887
✅ Saved best model!


...

Train Acc: 97.9904, Loss: 0.2361
[Epoch 98] lambda_mga: 0.4880
Val Acc: 0.8237


[Epoch 99]: 100%|██████████| 467/467 [00:14<00:00, 32.37it/s]


Train Acc: 98.6066, Loss: 0.2257
[Epoch 99] lambda_mga: 0.4920
Val Acc: 0.5425


[Epoch 100]: 100%|██████████| 467/467 [00:14<00:00, 32.71it/s]


Train Acc: 98.1779, Loss: 0.2302
[Epoch 100] lambda_mga: 0.4960
Val Acc: 0.5413

📊 Test Evaluation:
✅ Test Accuracy         : 87.12%
🎯 AUC                   : 0.9110
📌 Precision             : 0.9011
📌 Recall (Sensitivity)  : 0.9029
📌 Specificity           : 0.8109
📌 F1 Score              : 0.9020
📌 Balanced Accuracy     : 0.8569
📌 MCC                   : 0.7144

📌 Confusion Matrix:
[[223  52]
 [ 51 474]]

📁 테스트 지표 저장 완료: logs/final_test_metrics.csv


In [ ]:
# ✅ Test Accuracy         : 85.50%
# 🎯 AUC                   : 0.9074
# 📌 Precision             : 0.9034
# 📌 Recall (Sensitivity)  : 0.8724
# 📌 Specificity           : 0.8218
# 📌 F1 Score              : 0.8876
# 📌 Balanced Accuracy     : 0.8471
# 📌 MCC                   : 0.6844

# 📌 Confusion Matrix:
# [[226  49]
#  [ 67 458]]

# r18_cbam_mga_aug_lr4_ep100_weight2.pth
# 전체코드: ResNet18 + CBAM + MGA Loss + Lambda Scheduling (CE Weight + mask 회전)

import os, re, numpy as np, torch, gc
import csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import pandas as pd
import cv2
import torchvision.transforms as transforms
from PIL import Image
from datetime import datetime
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random

# -------------------- 디바이스 설정 --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------- 하이퍼파라미터 설정 --------------------
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"

batch_size = 8
num_epochs = 100
learning_rate = 1e-4

# lambda MGA 스케줄 설정
initial_lambda = 0.1
final_lambda = 0.5
total_epochs = num_epochs

# -------------------- Transform --------------------
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.CoarseDropout(
        max_holes=1, max_height=32, max_width=32, min_holes=1, 
        min_height=16, min_width=16, fill_value=0, p=0.5),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        file_path = self.paths[idx]
        label = self.labels[idx]
        fname = os.path.basename(file_path)

        img = np.load(file_path)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = img.astype(np.float32)

        if fname in bbox_dict:
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(img.shape[0], img.shape[1])).sum(axis=0)
        else:
            mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)

        if self.transform:
            transformed = self.transform(image=img, mask=mask)
            img = transformed['image']
            mask = transformed['mask']
        else:
            img = torch.tensor(img).unsqueeze(0)
            mask = torch.tensor(mask)

        return img, torch.tensor(label).long(), mask

    def __len__(self):
        return len(self.paths)

# 시드 고정
def seed_everything(seed=42):
    random.seed(seed)                         # 파이썬 random
    np.random.seed(seed)                      # numpy
    torch.manual_seed(seed)                   # torch CPU
    torch.cuda.manual_seed(seed)              # torch GPU
    torch.cuda.manual_seed_all(seed)          # multi-GPU
    torch.backends.cudnn.deterministic = True # 연산 동일하게
    torch.backends.cudnn.benchmark = False    # 연산 속도 최적화 OFF (같은 연산 보장)

# -------------------- Bounding Box Mask --------------------
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    masks = []
    for bbox in bbox_list:
        mask = np.zeros(image_size, dtype=np.float32)
        x_min, y_min, x_max, y_max = bbox
        mask[y_min:y_max, x_min:x_max] = 1.0
        masks.append(mask)
    masks = np.stack(masks)
    masks = np.expand_dims(masks, axis=1)
    return torch.tensor(masks, dtype=torch.float32)

# -------------------- Bounding Box CSV --------------------
def load_bbox_dict(csv_path):
    df = pd.read_csv(csv_path)
    bbox_dict = {}
    for _, row in df.iterrows():
        pid = row['pid']
        slice_idx = int(re.findall(r'\d+', str(row['slice']))[0])
        fname = f"{pid}_slice{slice_idx:04d}.npy"
        bbox = eval(row['bb'])
        bbox_dict.setdefault(fname, []).append(bbox)
    return bbox_dict

bbox_dict = load_bbox_dict(bbox_csv_path)

def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except:
        return None

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname): # fname : 파일 이름 (예: "LIDC-IDRI-1012_slice0039_5.npy")
    # 이 이름에서 malignancy score(악성도 점수)를 추출해서 라벨로 변환
    try:    # 파일명이 이상하거나 에러나면 except로 빠져나가서 None 반환함 (안전장치)
        score = int(fname.split("_")[-1].replace(".npy", ""))
        # 파일명에서 _ 제외하고 나머지 것들 중에 마지막에꺼를 가져와서 .npy를 "" 이렇게 공백으로 처리함
        # fname.split("_") -> ['LIDC-IDRI-1012', 'slice0039', '5.npy]
        # [-1] -> '5.npy'
        # .replace(".npy", "") -> '5'
        # int(...) -> 5 <- 이게 malignancy score
        return None if score == 3 else int(score >= 4)
        # 라벨 결정 로직으로
        # score == 3 -> 중립 -> None 반환 -> 학습에서 제외
        # score >= 4 -> 암(양성) -> 1
        # score <= 2 -> 정상(음성) -> 0
        # int(score >= 4)는 파이썬에서 True -> 1
        # False -> 0 이니깐 자동으로 라벨이 됨
    except:
        return None
        # 혹시 split이나 replace, int 변환이 실패하면 그냥 None 반환하고 무시

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    # PyTorch의 Dataset 클래스를 상속해서 커ㅡ텀 데이터셋 정의
    # 나중에 DataLoader랑 같이 쓰이기 때문에 __len__()이랑 __getitem__()을 꼭 넣어줘야함
    def __init__(self, paths, labels, transform=None):  # 생성자 : 세개의 인자를 받음
        # paths : 이미지 .npy 파일 경로 리스트
        # labels : 각 이미지에 대한 라벨 리스트 (0, 1 or None)
        # transform : 이미지 증강 설정 (train_transform, val_transform 등)
        self.paths = paths
        self.labels = labels
        self.transform = transform
        # 받은 인자를 멤버 변수로 저장. 나중에 gettem()에서 접근함

    def __getitem__(self, idx): # DataLoader가 이걸 호출할 때 index에 해당하는 sample 하나를 반환
        # 이미지, 라벨, 마스크( = MGA용 target) 3개를 리턴함
        file_path = self.paths[idx] # 파일 경로 불러오기
        label = self.labels[idx]    # 라벨 불러오기
        fname = os.path.basename(file_path) # 전체 경로에서 파일 이름만 추출 -> 나중에 bbox_dict[fname] 찾을때 쓰임

        img = np.load(file_path)    # .npy 파일에서 CT 슬라이스 불러오기 -> 흑백 CT 이미지, shape은 (H, W)
        img = np.clip(img, -1000, 400)  # CT 이미지 HU 값이 너무 크거나 작으면 노이즈 -> -1000(공기) ~ 400(연조직)으로 클리핑해서 노이즈 제거
        img = (img + 1000) / 1400.  # 정규화 : -1000 -> 0, 400 -> 1 사이 값으로 바꿔줌 -> 모델이 안정적으로 학습할 수 있도록 함
        img = img.astype(np.float32)
        
        h, w = img.shape
        img = np.expand_dims(img, axis=-1)  # (H, W, 1)

        # 마스크 생성
        if fname in bbox_dict:
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(h, w)).sum(dim=0).numpy()
        else:
            mask = np.zeros((h, w), dtype=np.float32)

        # transform 적용
        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img = augmented['image']  # shape: [1, 224, 224]
            mask = augmented['mask']  # shape: [224, 224]
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
            mask = torch.tensor(mask, dtype=torch.float32)

        return img, torch.tensor(label).long(), mask
        
    def __len__(self):
        return len(self.paths)
    # 전체 데이터셋 길이 반환 -> DataLoader가 아라야 배치 쪼갤 수 있음.

# -------------------- CBAM 정의 (MGA 포함) --------------------
# 2 Step : Channel Attention(어떤 채널에 집중할지) * Spatial Attention(어디에 집중할지) = 최종 Attention

class ChannelAttention(nn.Module):  # 입력 feature map의 채널별 중요도를 계산해서 강조함
    def __init__(self, planes, ratio=16):
        # planes : 입력 채널 수
        # ratio : 중간 채널 축소 비율. 기본 1/16으로 bottlenck 구성
        super().__init__()

        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        # MLP 역할을 하는 1x1 conv 블록 -> 채널 압축 -> 비선형 -> 복원 (shared는 avg/max 둘다에서 같이 씀)

        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()
        # 평균 풀링 / 최대 풀링으로 두가지 전역 정보를 추출
        # 마지막 sigmoid는 attention weight로 스케일링

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))
    # avg & max 풀링 경과를 각각 shape MLP에 통과시키고, 더한 후 sigmoid
    # -> shape : [B, C, 1, 1]
    # -> 채널마다 중요도 weight를 곱하게 됨

class SpatialAttention(nn.Module):  # 공간적으로 어디에 집중할지를 결정 -> 각 채널 내부에서 중요한 위치 찾기

    def __init__(self, k=7):    
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    # 채널 차원은 평균, 최대 두 개만 써서 concat
    # 그걸 1채널로 줄여주는 conv
    # 커널 크기 k=7이면 넓은 영역까지 감지 가능

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))
    # 입력 feature map에서 :
    # 평균, 최대값을 각 spatial 위치별로 구함 -> [B, 1, H, W] 두 개
    # concat -> [B, 2, H, w]
    # conv + sigmoid -> 위치별 중요도 map

class CBAM(nn.Module):  
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()
        self.last_attention = None
    # ChannelAttention, SpatialAttention을 내부에 선언
    # MGA를 위해 마지막 attention map을 저장하는 변수 포함

    def forward(self, x):
        ca_out = self.ca(x) * x
        sa_out = self.sa(ca_out)
        self.last_attention = sa_out
        return sa_out * ca_out
    # 채널 중요도 -> 곱함
    # 위치 중요도 -> 곱함
    # 둘 다 반영된 최종 feature map 리턴

# -------------------- ResNet18 + CBAM 모델 정의 --------------------
# BasicBlockCBAM : ResNet의 기본 Residual Block 하나를 정의
# → conv → BN → ReLU → conv → BN → (CBAM optional) → Add → ReLU

# ResNet18_CBAM : ResNet18 구조로 전체 네트워크 쌓기
# → conv1 → layer1~3 → layer4 → avgpool → fc

class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        # 입력 채널: in_planes, 출력 채널: out_planes, 3x3 커널, padding=1로 크기 유지, stride로 크기 조절
        self.bn1 = nn.BatchNorm2d(out_planes)   # 배치 정규화
        self.relu = nn.ReLU()   # 비선형 활성화 함수

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        # 두번째 conv, 채널 수 유지, 크기 유지
        self.bn2 = nn.BatchNorm2d(out_planes)   # 배치 정규화

        self.cbam = CBAM(out_planes) if use_cbam else None  # CBAM 모듈 사용 여부
        self.downsample = downsample    # residual 연결 시 차원 맞추는 conv

    def forward(self, x):
        residual = x    # skip connection용 입력 저장

        out = self.conv1(x) # 첫 번째 conv
        out = self.bn1(out) # 정규화
        out = self.relu(out)  # 활성화

        out = self.conv2(out)   # 두 번째 conv
        out = self.bn2(out) # 정규화

        if self.cbam:
            out = self.cbam(out)    # CBAM 적용

        if self.downsample:
            residual = self.downsample(x)   # shortcut 경로 보정

        out += residual # skip connection
        out = self.relu(out)    # 출력에 ReLU 적용

        return out  # 결과 반환

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64 # 조기 입력 채널 수 설정

        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # 입력: [B, 1, 224, 224] -> 출력: [B, 64, 112, 112], 큰 커널로 넓은 영역 캡처 
        self.bn1 = nn.BatchNorm2d(64)   # 정규화
        self.relu = nn.ReLU()   # 활성화

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # 풀링: [B, 64, 112, 112] -> [B, 64, 56, 56]

        self.layer1 = self._make_layer(64, blocks=2)  # [B, 64, 56, 56] 유지
        self.layer2 = self._make_layer(128, blocks=2, stride=2)  # [B, 128, 28, 28] 다운샘플링
        self.layer3 = self._make_layer(256, blocks=2, stride=2)  # [B, 256, 14, 14] 다운샘플링
        self.layer4 = self._make_layer(512, blocks=2, stride=2, use_cbam=False)  # [B, 512, 7, 7], CBAM 미사용

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # [B, 512, 7, 7] -> [B, 512, 1, 1]
        self.fc = nn.Linear(512, num_classes)  # [B, 512] -> [B, num_classes]

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        # Planes : 해당 레이어의 출력 채널 수
        # # blocks : 블록 수
        # stride=2인 경우 다운샘플링 (해상도 절반)

        downsample = None   # 스킵 연결해서 입력/출력 크기가 다르면 맞춰야 함

        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                # 1x1 conv로 채널 수   및 공간 크기 맞춤
                nn.BatchNorm2d(planes))
            
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        # 첫 블록은 다운샘플링 적용 가능성 있음
        self.in_planes = planes # 이후 블록을 위한 입력 채널 업데이트

        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
            # 나머지 블록은 stride=1로 동일한 해상도 유지

        return nn.Sequential(*layers)   # 블록들을 Seguential로 묶어 반환

    def forward(self, x):
        x = self.conv1(x)  # 입력: [B, 1, 224, 224] -> [B, 64, 112, 112]
        x = self.bn1(x)    # 정규화
        x = self.relu(x)   # ReLU 활성화
        x = self.maxpool(x)  # [B, 64, 112, 112] -> [B, 64, 56, 56]

        x = self.layer1(x)  # [B, 64, 56, 56]
        x = self.layer2(x)  # [B, 128, 28, 28]
        x = self.layer3(x)  # [B, 256, 14, 14]
        x = self.layer4(x)  # [B, 512, 7, 7]

        x = self.avgpool(x)  # [B, 512, 1, 1]
        x = torch.flatten(x, 1)  # [B, 512]
        x = self.fc(x)  # [B, num_classes]

        return x


# -------------------- 학습 루프 --------------------
# -------------------- run 함수 --------------------
def run():
    seed_everything(42)
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)

    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_dataset = CTDataset(train_files, train_labels, transform=train_transform)
    val_dataset = CTDataset(val_files, val_labels, transform=val_transform)
    test_dataset = CTDataset(test_files, test_labels, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # 모델, 손실함수, 옵티마이저 정의
    model = ResNet18_CBAM().to(device)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.6653, 0.3347], device=device))
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0  # 가장 높은 val accuracy를 저장
    save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "r18_cbam_mga_aug_lr4_ep100_weight2.pth")

    # 학습 루프 시작
    for epoch in range(num_epochs):
        # MGA 스케쥴링: 초기 lambda -> 점점 증가시킴
        lambda_mga = initial_lambda + (final_lambda - initial_lambda) * (epoch / total_epochs)

        model.train()  # 학습 모드로 변경
        epoch_loss = 0
        correct = 0
        total = 0

        # 한 epoch 동안 모든 train 데이터를 학습
        for images, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images = images.to(device)
            labels = labels.to(device)
            masks = masks.to(device)

            outputs = model(images)  # forward pass
            ce_loss = criterion(outputs, labels)  # cross entropy loss

            # -------------------- MGA Loss 계산 위치 --------------------
            attn_map = model.layer3[1].cbam.last_attention  # attention map 꺼내오기

            if attn_map is not None:
                attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)
                attn_loss = F.mse_loss(attn_map, masks)  # mask와의 MSE loss
                loss = ce_loss + lambda_mga * attn_loss
            else:
                loss = ce_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()

        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")
        print(f"[Epoch {epoch+1}] lambda_mga: {lambda_mga:.4f}")

        torch.cuda.empty_cache(); gc.collect()  # 메모리 정리

        # -------------------- 검증 --------------------
        model.eval()
        correct = 0; total = 0

        with torch.no_grad():
            for iamegs, labels, masks in val_loader:
                iamegs, labels, masks = iamegs.to(device), labels.to(device), masks.to(device)
                outputs = model(iamegs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # -------------------- 테스트 --------------------
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()

    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for images, labels, _ in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    # numpy 배열로 변환
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_probs = np.array(y_probs)

    # 지표 계산
    from sklearn.metrics import (
        classification_report, roc_auc_score, confusion_matrix,
        precision_score, recall_score, balanced_accuracy_score,
        matthews_corrcoef, f1_score
    )

    acc = (y_pred == y_true).mean()
    auc = roc_auc_score(y_true, y_probs)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    specificity = tn / (tn + fp + 1e-6)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    # 📋 출력
    print(f"✅ Test Accuracy         : {acc*100:.2f}%")
    print(f"🎯 AUC                   : {auc:.4f}")
    print(f"📌 Precision             : {precision:.4f}")
    print(f"📌 Recall (Sensitivity)  : {recall:.4f}")
    print(f"📌 Specificity           : {specificity:.4f}")
    print(f"📌 F1 Score              : {f1:.4f}")
    print(f"📌 Balanced Accuracy     : {balanced_acc:.4f}")
    print(f"📌 MCC                   : {mcc:.4f}")
    print("\n📌 Confusion Matrix:")
    print(cm)

    # 📁 CSV로 저장
    test_metrics = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "model": "ResNet18_CBAM_MGA",
        "phase": "test",
        "accuracy": round(acc, 4),
        "auc": round(auc, 4),
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "specificity": round(specificity, 4),
        "f1_score": round(f1, 4),
        "balanced_acc": round(balanced_acc, 4),
        "mcc": round(mcc, 4),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }

    csv_path = "logs/final_test_metrics.csv"
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    file_exists = os.path.exists(csv_path)

    with open(csv_path, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=test_metrics.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(test_metrics)

    print(f"\n📁 테스트 지표 저장 완료: {csv_path}")

# 진입점
if __name__ == "__main__":
    run()


In [ ]:
# 베스트 모델 시드 고정

# 전체코드: ResNet18 + CBAM + MGA Loss + Lambda Scheduling (CE Weight ver)

import os, re, numpy as np, torch, gc
import csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import pandas as pd
import cv2
import torchvision.transforms as transforms
from PIL import Image
from datetime import datetime
import random


# -------------------- 디바이스 설정 --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------- 하이퍼파라미터 설정 --------------------
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"

batch_size = 16
num_epochs = 100
learning_rate = 1e-4

# lambda MGA 스케줄 설정
initial_lambda = 0.1
final_lambda = 0.5
total_epochs = num_epochs

# -------------------- Transform --------------------
train_transform = transforms.Compose([
    transforms.ToPILImage(),    # numpy or tensor 이미지를 PIL 이미지 객체로 변환
    transforms.Resize((224, 224)),  # 이미지를 224x224로 resize
    transforms.RandomHorizontalFlip(),  # 이미지를 50% 확률로 좌우 반전
    transforms.RandomRotation(10),  # 이미지를 -10도 ~ +10도 사이로 랜덤 회전, 촬영 자세나 기울어짐에 대한 회전 강건성확보
    transforms.ToTensor(),  # PIL이미지 -> PyTorch Tensor로 변환, (H, W, C) -> (C, H, W), 값도 0255 -> 01 사이즈로 스케일 조정
    transforms.Normalize([0.5], [0.5]), # 평균 0.5, 표준편차 0.5로 정규화 -> 결과적으로 01 -> 11로 바뀜
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])  # 전체 이미지의 일부분 지우고 0으로 채움 (검은 사각형 생김)
    # p=0.5 : 50% 확률로 이 증강 적용
    # scale : 전체 이미지 대비 삭제 영역의 크기 비율
    # ratio : 지우는 사각형의 가로:세로 비율 범위
    # value=0 : 지운 곳을 검은색(0)으로 덮음
    # 폐 CT에서 병변이 항상 일정한 위치에 나오지 않으니까 모델이 특정 위치에 과적합되는걸 방지함 (overfitting 예방)

# 검증/테스트는 모델이 학습하지 않은 깨긋한 상태의 이미지로 정확도를 확인하기 위해서 검증용은 깔끔하게
val_transform = transforms.Compose([
    transforms.ToPILImage(),    # PIL 이미지 객체로 변환
    transforms.Resize((224, 224)),  # 사이즈 맞추기
    transforms.ToTensor(),  # PIL 이미지 -> PyTorch Tensor로 변환
    transforms.Normalize([0.5], [0.5])  # 정규화
])

# 시드 고정
def seed_everything(seed=42):
    random.seed(seed)                         # 파이썬 random
    np.random.seed(seed)                      # numpy
    torch.manual_seed(seed)                   # torch CPU
    torch.cuda.manual_seed(seed)              # torch GPU
    torch.cuda.manual_seed_all(seed)          # multi-GPU
    torch.backends.cudnn.deterministic = True # 연산 동일하게
    torch.backends.cudnn.benchmark = False    # 연산 속도 최적화 OFF (같은 연산 보장)

# -------------------- Bounding Box를 Binary Mask로 --------------------
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    # bbox_list : 한 이미지에 들어있는 bounding box 리스트
    # image_size : 출력할 마스크 크기. 보통 이미지와 동일한 (height, width) -> 디폴트는 224x224
    # bbox들을 binary mask로 바꿔주는 함수
    masks = []  # 여러 개의 bbox가 들어오니까, 각각의 마스크를 하나씩 리스트에 쌓기 위한 빈 리스트
    for bbox in bbox_list:  # bbox_list를 하나씩 돌면서 처리 -> [x_min, y_min, x_max, y_max]네 좌표로 구성된 하나의 사각형 영역 
        mask = np.zeros(image_size, dtype=np.float32)   # 224x224짜리 0으로 꽉 찬 2D 배열을 하나 생성
        # 배경이 흰 종이를 만드는 느낌으로 만들고, 사각 영역만 1로 덧칠할거임
        x_min, y_min, x_max, y_max = bbox   # 각 bbox의 네 좌표값을 각각 변수로 언팩. -> 마스크의 해당 영역에 사각형을 칠하기 위해서
        mask[y_min:y_max, x_min:x_max] = 1.0    # y_min, y_max, x_min, x_max까지의 범위에 1.0을 채워 넣음
        # -> 마스크에서 bbox에 해당하는 사각형 영역만 1(foreground)로 표시됨. 나머진 여전히 0(background)
        masks.append(mask)  # 지금 만든 마스크(2D 배열)를 리스트에 추가 -> [mask1, mask2, ...]이렇게 쌓임

    masks = np.stack(masks) # 리스트를 하나의 3D 배열로 합침 -> shape : [N, H, W] -> N은 bbox 개수
    masks = np.expand_dims(masks, axis=1)   # 텐서 shape을 [N, 1, H, W]로 바꿈
    # PyTorch 모델에서 기대하는 (batch x channel x height x width) 포맷 맞추기

    return torch.tensor(masks, dtype=torch.float32)
    # numpy 배열을 PyTorch 텐서로 변환해서 리턴

    # 한 bbox → 하나의 마스크 → 여러 개면 쌓아서 batch 형태로
# -------------------- Bounding Box CSV 로드 --------------------
def load_bbox_dict(csv_path):
    # csv_path : bounding box 정보가 들어있는 CSV 파일 경로
    # 반환값 : {filename:[bbox1, bbox2, ...]} 형태의 딕셔너리
    df = pd.read_csv(csv_path)  # CSV파일을 pandas DataFrame으로 읽어옴
    bbox_dict = {}
    # key : 슬라이스 파일 이름 (ex. "LIDC-IDRI-1012_slice0004.npy")
    # value : 해당 슬라이스에 존재하는 bbox들의 리스트
    for _, row in df.iterrows():    # DataFrame의 모든 행(row)를 하나씩 순회
        # row는 한 줄(=한 bbox)의 정보를 담고 있음

        pid = row['pid']    # 환자 ID (예: "LIDC-IDRI-1012") -> 이미지 이름 구성 요소
        slice_str = row['slice']    # 슬라이스 정보가 들어있는 문자열 (예: "slice_0039")
        slice_idx = int(re.findall(r'\d+', str(slice_str))[0])  # re.findall()로 문자열에서 숫자만 뽑아냄
        # "slice_0039" -> ['0039'] -> [0] -> 39 (슬라이스 번호를 정수로 추출함)
        fname = f"{pid}_slice{slice_idx:04d}.npy"   # 파일명 구성 (예: "LIDC-IDRI-1012_slice0039.npy")
        # {:04d}는 4자리 정수로 만들고 빈자리는 0으로 채워줌 (39 -> 0039)
        bbox = eval(row['bb'])  # row['bb']는 문자열 형태의 bbox (예: "[20, 30, 80, 100]")
        # eval()을 써서 문자열을 리스트로 바꿔줌
        # 주의 : 보안 상 위험할 수 있는 함수지만, 여긴 내부 데이터라 사용중
        bbox_dict.setdefault(fname, []).append(bbox)    # fname이라는 key가 딕셔너리에 없으면 []로 초기화하고,
        # 거기에 bbox를 append -> 슬라이스 하나에 bbox 여러개 있어도 전부 리스트로 모아줌
    return bbox_dict    # 최종적으로 {filename: [bbox1, bbox2, ...]} 형태의 딕셔너리 반환

bbox_dict = load_bbox_dict(bbox_csv_path)
# 실제로 csv_path에 있는 정보를 불러와서 bbox_dict에 저장함
# 이걸 나주에 Dataset 클래스에서 fname 기준을 꺼내쓰게 됨

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname): # fname : 파일 이름 (예: "LIDC-IDRI-1012_slice0039_5.npy")
    # 이 이름에서 malignancy score(악성도 점수)를 추출해서 라벨로 변환
    try:    # 파일명이 이상하거나 에러나면 except로 빠져나가서 None 반환함 (안전장치)
        score = int(fname.split("_")[-1].replace(".npy", ""))
        # 파일명에서 _ 제외하고 나머지 것들 중에 마지막에꺼를 가져와서 .npy를 "" 이렇게 공백으로 처리함
        # fname.split("_") -> ['LIDC-IDRI-1012', 'slice0039', '5.npy]
        # [-1] -> '5.npy'
        # .replace(".npy", "") -> '5'
        # int(...) -> 5 <- 이게 malignancy score
        return None if score == 3 else int(score >= 4)
        # 라벨 결정 로직으로
        # score == 3 -> 중립 -> None 반환 -> 학습에서 제외
        # score >= 4 -> 암(양성) -> 1
        # score <= 2 -> 정상(음성) -> 0
        # int(score >= 4)는 파이썬에서 True -> 1
        # False -> 0 이니깐 자동으로 라벨이 됨
    except:
        return None
        # 혹시 split이나 replace, int 변환이 실패하면 그냥 None 반환하고 무시

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    # PyTorch의 Dataset 클래스를 상속해서 커ㅡ텀 데이터셋 정의
    # 나중에 DataLoader랑 같이 쓰이기 때문에 __len__()이랑 __getitem__()을 꼭 넣어줘야함
    def __init__(self, paths, labels, transform=None):  # 생성자 : 세개의 인자를 받음
        # paths : 이미지 .npy 파일 경로 리스트
        # labels : 각 이미지에 대한 라벨 리스트 (0, 1 or None)
        # transform : 이미지 증강 설정 (train_transform, val_transform 등)
        self.paths = paths
        self.labels = labels
        self.transform = transform
        # 받은 인자를 멤버 변수로 저장. 나중에 gettem()에서 접근함

    def __getitem__(self, idx): # DataLoader가 이걸 호출할 때 index에 해당하는 sample 하나를 반환
        # 이미지, 라벨, 마스크( = MGA용 target) 3개를 리턴함
        file_path = self.paths[idx] # 파일 경로 불러오기
        label = self.labels[idx]    # 라벨 불러오기
        fname = os.path.basename(file_path) # 전체 경로에서 파일 이름만 추출 -> 나중에 bbox_dict[fname] 찾을때 쓰임

        img = np.load(file_path)    # .npy 파일에서 CT 슬라이스 불러오기 -> 흑백 CT 이미지, shape은 (H, W)
        img = np.clip(img, -1000, 400)  # CT 이미지 HU 값이 너무 크거나 작으면 노이즈 -> -1000(공기) ~ 400(연조직)으로 클리핑해서 노이즈 제거
        img = (img + 1000) / 1400.  # 정규화 : -1000 -> 0, 400 -> 1 사이 값으로 바꿔줌 -> 모델이 안정적으로 학습할 수 있도록 함
        img = np.expand_dims(img, axis=-1)  # CT는 채널이 1개니깐 (H, W) -> (H, w, 1)로 바꿔줌
        # 나중에 PyTorch에서 (C, H, W)로 바꾸기 위함

        if self.transform:  # 데이터 증강(transform)이 있다면 적용
            img = self.transform(img)   
        else:   # 없으면 numpy -> tensor 변환하고 (H, W, C) -> (C, H, W)로 순서 바꿈
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)

        if fname in bbox_dict:  # 이 이미지에 bbox가 존재하면 -> 마스크 생성
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(224, 224))
            # image_size는 transform과 동일하게 224x224
        else:   # bbox가 없다면 전부 0으로 채워진 마스크 생성 -> MGA Loss 계산 시 참고용으로 쓰일 수 있음
            mask = torch.zeros((1, 224, 224), dtype=torch.float32)

        return img, torch.tensor(label).long(), mask.squeeze(0)
        # 반환값 3개 :
        # img : shape[1, 224, 224]
        # label : int(0 or 1)
        # mask : [224, 224] <- squeeze로 채널 1개 제거

    def __len__(self):
        return len(self.paths)
    # 전체 데이터셋 길이 반환 -> DataLoader가 아라야 배치 쪼갤 수 있음.

# -------------------- CBAM 정의 (MGA 포함) --------------------
# 2 Step : Channel Attention(어떤 채널에 집중할지) * Spatial Attention(어디에 집중할지) = 최종 Attention

class ChannelAttention(nn.Module):  # 입력 feature map의 채널별 중요도를 계산해서 강조함
    def __init__(self, planes, ratio=16):
        # planes : 입력 채널 수
        # ratio : 중간 채널 축소 비율. 기본 1/16으로 bottlenck 구성
        super().__init__()

        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        # MLP 역할을 하는 1x1 conv 블록 -> 채널 압축 -> 비선형 -> 복원 (shared는 avg/max 둘다에서 같이 씀)

        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()
        # 평균 풀링 / 최대 풀링으로 두가지 전역 정보를 추출
        # 마지막 sigmoid는 attention weight로 스케일링

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))
    # avg & max 풀링 경과를 각각 shape MLP에 통과시키고, 더한 후 sigmoid
    # -> shape : [B, C, 1, 1]
    # -> 채널마다 중요도 weight를 곱하게 됨

class SpatialAttention(nn.Module):  # 공간적으로 어디에 집중할지를 결정 -> 각 채널 내부에서 중요한 위치 찾기

    def __init__(self, k=7):    
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    # 채널 차원은 평균, 최대 두 개만 써서 concat
    # 그걸 1채널로 줄여주는 conv
    # 커널 크기 k=7이면 넓은 영역까지 감지 가능

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))
    # 입력 feature map에서 :
    # 평균, 최대값을 각 spatial 위치별로 구함 -> [B, 1, H, W] 두 개
    # concat -> [B, 2, H, w]
    # conv + sigmoid -> 위치별 중요도 map

class CBAM(nn.Module):  
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()
        self.last_attention = None
    # ChannelAttention, SpatialAttention을 내부에 선언
    # MGA를 위해 마지막 attention map을 저장하는 변수 포함

    def forward(self, x):
        ca_out = self.ca(x) * x
        sa_out = self.sa(ca_out)
        self.last_attention = sa_out
        return sa_out * ca_out
    # 채널 중요도 -> 곱함
    # 위치 중요도 -> 곱함
    # 둘 다 반영된 최종 feature map 리턴

# -------------------- ResNet18 + CBAM 모델 정의 --------------------
# BasicBlockCBAM : ResNet의 기본 Residual Block 하나를 정의
# → conv → BN → ReLU → conv → BN → (CBAM optional) → Add → ReLU

# ResNet18_CBAM : ResNet18 구조로 전체 네트워크 쌓기
# → conv1 → layer1~3 → layer4 → avgpool → fc

class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        # 입력 채널: in_planes, 출력 채널: out_planes, 3x3 커널, padding=1로 크기 유지, stride로 크기 조절
        self.bn1 = nn.BatchNorm2d(out_planes)   # 배치 정규화
        self.relu = nn.ReLU()   # 비선형 활성화 함수

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        # 두번째 conv, 채널 수 유지, 크기 유지
        self.bn2 = nn.BatchNorm2d(out_planes)   # 배치 정규화

        self.cbam = CBAM(out_planes) if use_cbam else None  # CBAM 모듈 사용 여부
        self.downsample = downsample    # residual 연결 시 차원 맞추는 conv

    def forward(self, x):
        residual = x    # skip connection용 입력 저장

        out = self.conv1(x) # 첫 번째 conv
        out = self.bn1(out) # 정규화
        out = self.relu(out)  # 활성화

        out = self.conv2(out)   # 두 번째 conv
        out = self.bn2(out) # 정규화

        if self.cbam:
            out = self.cbam(out)    # CBAM 적용

        if self.downsample:
            residual = self.downsample(x)   # shortcut 경로 보정

        out += residual # skip connection
        out = self.relu(out)    # 출력에 ReLU 적용

        return out  # 결과 반환

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64 # 조기 입력 채널 수 설정

        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # 입력: [B, 1, 224, 224] -> 출력: [B, 64, 112, 112], 큰 커널로 넓은 영역 캡처 
        self.bn1 = nn.BatchNorm2d(64)   # 정규화
        self.relu = nn.ReLU()   # 활성화

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # 풀링: [B, 64, 112, 112] -> [B, 64, 56, 56]

        self.layer1 = self._make_layer(64, blocks=2)  # [B, 64, 56, 56] 유지
        self.layer2 = self._make_layer(128, blocks=2, stride=2)  # [B, 128, 28, 28] 다운샘플링
        self.layer3 = self._make_layer(256, blocks=2, stride=2)  # [B, 256, 14, 14] 다운샘플링
        self.layer4 = self._make_layer(512, blocks=2, stride=2, use_cbam=False)  # [B, 512, 7, 7], CBAM 미사용

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # [B, 512, 7, 7] -> [B, 512, 1, 1]
        self.fc = nn.Linear(512, num_classes)  # [B, 512] -> [B, num_classes]

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        # Planes : 해당 레이어의 출력 채널 수
        # # blocks : 블록 수
        # stride=2인 경우 다운샘플링 (해상도 절반)

        downsample = None   # 스킵 연결해서 입력/출력 크기가 다르면 맞춰야 함

        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                # 1x1 conv로 채널 수   및 공간 크기 맞춤
                nn.BatchNorm2d(planes))
            
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        # 첫 블록은 다운샘플링 적용 가능성 있음
        self.in_planes = planes # 이후 블록을 위한 입력 채널 업데이트

        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
            # 나머지 블록은 stride=1로 동일한 해상도 유지

        return nn.Sequential(*layers)   # 블록들을 Seguential로 묶어 반환

    def forward(self, x):
        x = self.conv1(x)  # 입력: [B, 1, 224, 224] -> [B, 64, 112, 112]
        x = self.bn1(x)    # 정규화
        x = self.relu(x)   # ReLU 활성화
        x = self.maxpool(x)  # [B, 64, 112, 112] -> [B, 64, 56, 56]

        x = self.layer1(x)  # [B, 64, 56, 56]
        x = self.layer2(x)  # [B, 128, 28, 28]
        x = self.layer3(x)  # [B, 256, 14, 14]
        x = self.layer4(x)  # [B, 512, 7, 7]

        x = self.avgpool(x)  # [B, 512, 1, 1]
        x = torch.flatten(x, 1)  # [B, 512]
        x = self.fc(x)  # [B, num_classes]

        return x


# -------------------- 학습 루프 --------------------
def run():
    seed_everything(42)
    # 모든 CT 슬라이스 파일 경로 불러오기 (LIDC-IDRI 환자 폴더 안의 .npy 파일들)
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))

    # 파일 경로와 해당 파일의 라벨을 튜플로 저장
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    # 라벨이 None이 아닌 데이터만 필터링 (중립 제외)
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]

    # 파일, 라벨을 리스트로 분리
    files, labels = zip(*file_label_pairs)

    # 전체 데이터를 train(70%), val(15%), test(15%)로 분할
    train_files, temp_files, train_labels, temp_labels = train_test_split(
    files, labels, test_size=0.3, random_state=42)

    val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=0.5, random_state=42)

    # 데이터 불러오기
    train_dataset = CTDataset(train_files, train_labels, transform=train_transform)
    val_dataset = CTDataset(val_files, val_labels, transform=val_transform)
    test_dataset = CTDataset(test_files, test_labels, transform=val_transform)

    # 데이터 로더
    train_loader = DataLoader(train_dataset,  batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # 모델, 손실함수, 옵티마이저 정의
    model = ResNet18_CBAM().to(device)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.6653, 0.3347], device=device))
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0  # 가장 높은 val accuracy를 저장
    save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "r18_cbam_mga_aug_lr4_ep100_weight_seedfix.pth")

    # 학습 루프 시작
    for epoch in range(num_epochs):
        # MGA 스케쥴링: 초기 lambda -> 점점 증가시킴
        lambda_mga = initial_lambda + (final_lambda - initial_lambda) * (epoch / total_epochs)

        model.train()  # 학습 모드로 변경
        epoch_loss = 0
        correct = 0
        total = 0

        # 한 epoch 동안 모든 train 데이터를 학습
        for images, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images = images.to(device)
            labels = labels.to(device)
            masks = masks.to(device)

            outputs = model(images)  # forward pass
            ce_loss = criterion(outputs, labels)  # cross entropy loss

            # -------------------- MGA Loss 계산 위치 --------------------
            attn_map = model.layer3[1].cbam.last_attention  # attention map 꺼내오기

            if attn_map is not None:
                attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)
                attn_loss = F.mse_loss(attn_map, masks)  # mask와의 MSE loss
                loss = ce_loss + lambda_mga * attn_loss
            else:
                loss = ce_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()

        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")
        print(f"[Epoch {epoch+1}] lambda_mga: {lambda_mga:.4f}")

        torch.cuda.empty_cache(); gc.collect()  # 메모리 정리

        # -------------------- 검증 --------------------
        model.eval()
        correct = 0; total = 0

        with torch.no_grad():
            for iamegs, labels, masks in val_loader:
                iamegs, labels, masks = iamegs.to(device), labels.to(device), masks.to(device)
                outputs = model(iamegs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # -------------------- 테스트 --------------------
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()

    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for images, labels, _ in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    # numpy 배열로 변환
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_probs = np.array(y_probs)

    # 지표 계산
    from sklearn.metrics import (
        classification_report, roc_auc_score, confusion_matrix,
        precision_score, recall_score, balanced_accuracy_score,
        matthews_corrcoef, f1_score
    )

    acc = (y_pred == y_true).mean()
    auc = roc_auc_score(y_true, y_probs)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    specificity = tn / (tn + fp + 1e-6)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    # 📋 출력
    print(f"✅ Test Accuracy         : {acc*100:.2f}%")
    print(f"🎯 AUC                   : {auc:.4f}")
    print(f"📌 Precision             : {precision:.4f}")
    print(f"📌 Recall (Sensitivity)  : {recall:.4f}")
    print(f"📌 Specificity           : {specificity:.4f}")
    print(f"📌 F1 Score              : {f1:.4f}")
    print(f"📌 Balanced Accuracy     : {balanced_acc:.4f}")
    print(f"📌 MCC                   : {mcc:.4f}")
    print("\n📌 Confusion Matrix:")
    print(cm)

    # 📁 CSV로 저장
    test_metrics = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "model": "ResNet18_CBAM_MGA",
        "phase": "test",
        "accuracy": round(acc, 4),
        "auc": round(auc, 4),
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "specificity": round(specificity, 4),
        "f1_score": round(f1, 4),
        "balanced_acc": round(balanced_acc, 4),
        "mcc": round(mcc, 4),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }

    csv_path = "logs/final_test_metrics.csv"
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    file_exists = os.path.exists(csv_path)

    with open(csv_path, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=test_metrics.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(test_metrics)

    print(f"\n📁 테스트 지표 저장 완료: {csv_path}")

# 진입점
if __name__ == "__main__":
    run()


Using device: cuda


[Epoch 1]: 100%|██████████| 234/234 [00:09<00:00, 25.49it/s]


Train Acc: 58.0386, Loss: 0.6966
[Epoch 1] lambda_mga: 0.1000
Val Acc: 0.4375
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:10<00:00, 23.19it/s]


Train Acc: 61.7631, Loss: 0.6565
[Epoch 2] lambda_mga: 0.1040
Val Acc: 0.5600
✅ Saved best model!


...

[Epoch 12]: 100%|██████████| 234/234 [00:12<00:00, 19.07it/s]


Train Acc: 81.6720, Loss: 0.4181
[Epoch 12] lambda_mga: 0.1440
Val Acc: 0.7200


[Epoch 13]: 100%|██████████| 234/234 [00:12<00:00, 19.12it/s]


Train Acc: 82.5295, Loss: 0.3966
[Epoch 13] lambda_mga: 0.1480
Val Acc: 0.7600
✅ Saved best model!


[Epoch 14]:  19%|█▉        | 44/234 [00:02<00:07, 25.48it/s]

In [ ]:
# 👍👍👍👍 ✅ Test Accuracy         : 90.62%
# 🎯 AUC                   : 0.9408
# 📌 Precision             : 0.9261
# 📌 Recall (Sensitivity)  : 0.9314
# 📌 Specificity           : 0.8582
# 📌 F1 Score              : 0.9288
# 📌 Balanced Accuracy     : 0.8948
# 📌 MCC                   : 0.7917

# 📌 Confusion Matrix:
# [[236  39]
#  [ 36 489]]
# r18_cbam_mga_aug_lr4_ep100_weight.pth
# 전체코드: ResNet18 + CBAM + MGA Loss (CE Weight ver)

import os, re, numpy as np, torch, gc
import csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import pandas as pd
import cv2
import torchvision.transforms as transforms
from PIL import Image
from datetime import datetime


# -------------------- 디바이스 설정 --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------- 하이퍼파라미터 설정 --------------------
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"

batch_size = 8
num_epochs = 100
learning_rate = 1e-4

# lambda MGA 스케줄 설정
initial_lambda = 0.1
final_lambda = 0.5
total_epochs = num_epochs

# -------------------- Transform --------------------
train_transform = transforms.Compose([
    transforms.ToPILImage(),    # numpy or tensor 이미지를 PIL 이미지 객체로 변환
    transforms.Resize((224, 224)),  # 이미지를 224x224로 resize
    transforms.RandomHorizontalFlip(),  # 이미지를 50% 확률로 좌우 반전
    transforms.RandomRotation(10),  # 이미지를 -10도 ~ +10도 사이로 랜덤 회전, 촬영 자세나 기울어짐에 대한 회전 강건성확보
    transforms.ToTensor(),  # PIL이미지 -> PyTorch Tensor로 변환, (H, W, C) -> (C, H, W), 값도 0255 -> 01 사이즈로 스케일 조정
    transforms.Normalize([0.5], [0.5]), # 평균 0.5, 표준편차 0.5로 정규화 -> 결과적으로 01 -> 11로 바뀜
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])  # 전체 이미지의 일부분 지우고 0으로 채움 (검은 사각형 생김)
    # p=0.5 : 50% 확률로 이 증강 적용
    # scale : 전체 이미지 대비 삭제 영역의 크기 비율
    # ratio : 지우는 사각형의 가로:세로 비율 범위
    # value=0 : 지운 곳을 검은색(0)으로 덮음
    # 폐 CT에서 병변이 항상 일정한 위치에 나오지 않으니까 모델이 특정 위치에 과적합되는걸 방지함 (overfitting 예방)

# 검증/테스트는 모델이 학습하지 않은 깨긋한 상태의 이미지로 정확도를 확인하기 위해서 검증용은 깔끔하게
val_transform = transforms.Compose([
    transforms.ToPILImage(),    # PIL 이미지 객체로 변환
    transforms.Resize((224, 224)),  # 사이즈 맞추기
    transforms.ToTensor(),  # PIL 이미지 -> PyTorch Tensor로 변환
    transforms.Normalize([0.5], [0.5])  # 정규화
])

# -------------------- Bounding Box를 Binary Mask로 --------------------
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    # bbox_list : 한 이미지에 들어있는 bounding box 리스트
    # image_size : 출력할 마스크 크기. 보통 이미지와 동일한 (height, width) -> 디폴트는 224x224
    # bbox들을 binary mask로 바꿔주는 함수
    masks = []  # 여러 개의 bbox가 들어오니까, 각각의 마스크를 하나씩 리스트에 쌓기 위한 빈 리스트
    for bbox in bbox_list:  # bbox_list를 하나씩 돌면서 처리 -> [x_min, y_min, x_max, y_max]네 좌표로 구성된 하나의 사각형 영역 
        mask = np.zeros(image_size, dtype=np.float32)   # 224x224짜리 0으로 꽉 찬 2D 배열을 하나 생성
        # 배경이 흰 종이를 만드는 느낌으로 만들고, 사각 영역만 1로 덧칠할거임
        x_min, y_min, x_max, y_max = bbox   # 각 bbox의 네 좌표값을 각각 변수로 언팩. -> 마스크의 해당 영역에 사각형을 칠하기 위해서
        mask[y_min:y_max, x_min:x_max] = 1.0    # y_min, y_max, x_min, x_max까지의 범위에 1.0을 채워 넣음
        # -> 마스크에서 bbox에 해당하는 사각형 영역만 1(foreground)로 표시됨. 나머진 여전히 0(background)
        masks.append(mask)  # 지금 만든 마스크(2D 배열)를 리스트에 추가 -> [mask1, mask2, ...]이렇게 쌓임

    masks = np.stack(masks) # 리스트를 하나의 3D 배열로 합침 -> shape : [N, H, W] -> N은 bbox 개수
    masks = np.expand_dims(masks, axis=1)   # 텐서 shape을 [N, 1, H, W]로 바꿈
    # PyTorch 모델에서 기대하는 (batch x channel x height x width) 포맷 맞추기

    return torch.tensor(masks, dtype=torch.float32)
    # numpy 배열을 PyTorch 텐서로 변환해서 리턴

    # 한 bbox → 하나의 마스크 → 여러 개면 쌓아서 batch 형태로
# -------------------- Bounding Box CSV 로드 --------------------
def load_bbox_dict(csv_path):
    # csv_path : bounding box 정보가 들어있는 CSV 파일 경로
    # 반환값 : {filename:[bbox1, bbox2, ...]} 형태의 딕셔너리
    df = pd.read_csv(csv_path)  # CSV파일을 pandas DataFrame으로 읽어옴
    bbox_dict = {}
    # key : 슬라이스 파일 이름 (ex. "LIDC-IDRI-1012_slice0004.npy")
    # value : 해당 슬라이스에 존재하는 bbox들의 리스트
    for _, row in df.iterrows():    # DataFrame의 모든 행(row)를 하나씩 순회
        # row는 한 줄(=한 bbox)의 정보를 담고 있음

        pid = row['pid']    # 환자 ID (예: "LIDC-IDRI-1012") -> 이미지 이름 구성 요소
        slice_str = row['slice']    # 슬라이스 정보가 들어있는 문자열 (예: "slice_0039")
        slice_idx = int(re.findall(r'\d+', str(slice_str))[0])  # re.findall()로 문자열에서 숫자만 뽑아냄
        # "slice_0039" -> ['0039'] -> [0] -> 39 (슬라이스 번호를 정수로 추출함)
        fname = f"{pid}_slice{slice_idx:04d}.npy"   # 파일명 구성 (예: "LIDC-IDRI-1012_slice0039.npy")
        # {:04d}는 4자리 정수로 만들고 빈자리는 0으로 채워줌 (39 -> 0039)
        bbox = eval(row['bb'])  # row['bb']는 문자열 형태의 bbox (예: "[20, 30, 80, 100]")
        # eval()을 써서 문자열을 리스트로 바꿔줌
        # 주의 : 보안 상 위험할 수 있는 함수지만, 여긴 내부 데이터라 사용중
        bbox_dict.setdefault(fname, []).append(bbox)    # fname이라는 key가 딕셔너리에 없으면 []로 초기화하고,
        # 거기에 bbox를 append -> 슬라이스 하나에 bbox 여러개 있어도 전부 리스트로 모아줌
    return bbox_dict    # 최종적으로 {filename: [bbox1, bbox2, ...]} 형태의 딕셔너리 반환

bbox_dict = load_bbox_dict(bbox_csv_path)
# 실제로 csv_path에 있는 정보를 불러와서 bbox_dict에 저장함
# 이걸 나주에 Dataset 클래스에서 fname 기준을 꺼내쓰게 됨

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname): # fname : 파일 이름 (예: "LIDC-IDRI-1012_slice0039_5.npy")
    # 이 이름에서 malignancy score(악성도 점수)를 추출해서 라벨로 변환
    try:    # 파일명이 이상하거나 에러나면 except로 빠져나가서 None 반환함 (안전장치)
        score = int(fname.split("_")[-1].replace(".npy", ""))
        # 파일명에서 _ 제외하고 나머지 것들 중에 마지막에꺼를 가져와서 .npy를 "" 이렇게 공백으로 처리함
        # fname.split("_") -> ['LIDC-IDRI-1012', 'slice0039', '5.npy]
        # [-1] -> '5.npy'
        # .replace(".npy", "") -> '5'
        # int(...) -> 5 <- 이게 malignancy score
        return None if score == 3 else int(score >= 4)
        # 라벨 결정 로직으로
        # score == 3 -> 중립 -> None 반환 -> 학습에서 제외
        # score >= 4 -> 암(양성) -> 1
        # score <= 2 -> 정상(음성) -> 0
        # int(score >= 4)는 파이썬에서 True -> 1
        # False -> 0 이니깐 자동으로 라벨이 됨
    except:
        return None
        # 혹시 split이나 replace, int 변환이 실패하면 그냥 None 반환하고 무시

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    # PyTorch의 Dataset 클래스를 상속해서 커ㅡ텀 데이터셋 정의
    # 나중에 DataLoader랑 같이 쓰이기 때문에 __len__()이랑 __getitem__()을 꼭 넣어줘야함
    def __init__(self, paths, labels, transform=None):  # 생성자 : 세개의 인자를 받음
        # paths : 이미지 .npy 파일 경로 리스트
        # labels : 각 이미지에 대한 라벨 리스트 (0, 1 or None)
        # transform : 이미지 증강 설정 (train_transform, val_transform 등)
        self.paths = paths
        self.labels = labels
        self.transform = transform
        # 받은 인자를 멤버 변수로 저장. 나중에 gettem()에서 접근함

    def __getitem__(self, idx): # DataLoader가 이걸 호출할 때 index에 해당하는 sample 하나를 반환
        # 이미지, 라벨, 마스크( = MGA용 target) 3개를 리턴함
        file_path = self.paths[idx] # 파일 경로 불러오기
        label = self.labels[idx]    # 라벨 불러오기
        fname = os.path.basename(file_path) # 전체 경로에서 파일 이름만 추출 -> 나중에 bbox_dict[fname] 찾을때 쓰임

        img = np.load(file_path)    # .npy 파일에서 CT 슬라이스 불러오기 -> 흑백 CT 이미지, shape은 (H, W)
        img = np.clip(img, -1000, 400)  # CT 이미지 HU 값이 너무 크거나 작으면 노이즈 -> -1000(공기) ~ 400(연조직)으로 클리핑해서 노이즈 제거
        img = (img + 1000) / 1400.  # 정규화 : -1000 -> 0, 400 -> 1 사이 값으로 바꿔줌 -> 모델이 안정적으로 학습할 수 있도록 함
        img = np.expand_dims(img, axis=-1)  # CT는 채널이 1개니깐 (H, W) -> (H, w, 1)로 바꿔줌
        # 나중에 PyTorch에서 (C, H, W)로 바꾸기 위함

        if self.transform:  # 데이터 증강(transform)이 있다면 적용
            img = self.transform(img)   
        else:   # 없으면 numpy -> tensor 변환하고 (H, W, C) -> (C, H, W)로 순서 바꿈
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)

        if fname in bbox_dict:  # 이 이미지에 bbox가 존재하면 -> 마스크 생성
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(224, 224))
            # image_size는 transform과 동일하게 224x224
        else:   # bbox가 없다면 전부 0으로 채워진 마스크 생성 -> MGA Loss 계산 시 참고용으로 쓰일 수 있음
            mask = torch.zeros((1, 224, 224), dtype=torch.float32)

        return img, torch.tensor(label).long(), mask.squeeze(0)
        # 반환값 3개 :
        # img : shape[1, 224, 224]
        # label : int(0 or 1)
        # mask : [224, 224] <- squeeze로 채널 1개 제거

    def __len__(self):
        return len(self.paths)
    # 전체 데이터셋 길이 반환 -> DataLoader가 아라야 배치 쪼갤 수 있음.

# -------------------- CBAM 정의 (MGA 포함) --------------------
# 2 Step : Channel Attention(어떤 채널에 집중할지) * Spatial Attention(어디에 집중할지) = 최종 Attention

class ChannelAttention(nn.Module):  # 입력 feature map의 채널별 중요도를 계산해서 강조함
    def __init__(self, planes, ratio=16):
        # planes : 입력 채널 수
        # ratio : 중간 채널 축소 비율. 기본 1/16으로 bottlenck 구성
        super().__init__()

        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        # MLP 역할을 하는 1x1 conv 블록 -> 채널 압축 -> 비선형 -> 복원 (shared는 avg/max 둘다에서 같이 씀)

        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()
        # 평균 풀링 / 최대 풀링으로 두가지 전역 정보를 추출
        # 마지막 sigmoid는 attention weight로 스케일링

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))
    # avg & max 풀링 경과를 각각 shape MLP에 통과시키고, 더한 후 sigmoid
    # -> shape : [B, C, 1, 1]
    # -> 채널마다 중요도 weight를 곱하게 됨

class SpatialAttention(nn.Module):  # 공간적으로 어디에 집중할지를 결정 -> 각 채널 내부에서 중요한 위치 찾기

    def __init__(self, k=7):    
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    # 채널 차원은 평균, 최대 두 개만 써서 concat
    # 그걸 1채널로 줄여주는 conv
    # 커널 크기 k=7이면 넓은 영역까지 감지 가능

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))
    # 입력 feature map에서 :
    # 평균, 최대값을 각 spatial 위치별로 구함 -> [B, 1, H, W] 두 개
    # concat -> [B, 2, H, w]
    # conv + sigmoid -> 위치별 중요도 map

class CBAM(nn.Module):  
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()
        self.last_attention = None
    # ChannelAttention, SpatialAttention을 내부에 선언
    # MGA를 위해 마지막 attention map을 저장하는 변수 포함

    def forward(self, x):
        ca_out = self.ca(x) * x
        sa_out = self.sa(ca_out)
        self.last_attention = sa_out
        return sa_out * ca_out
    # 채널 중요도 -> 곱함
    # 위치 중요도 -> 곱함
    # 둘 다 반영된 최종 feature map 리턴

# -------------------- ResNet18 + CBAM 모델 정의 --------------------
# BasicBlockCBAM : ResNet의 기본 Residual Block 하나를 정의
# → conv → BN → ReLU → conv → BN → (CBAM optional) → Add → ReLU

# ResNet18_CBAM : ResNet18 구조로 전체 네트워크 쌓기
# → conv1 → layer1~3 → layer4 → avgpool → fc

class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        # 입력 채널: in_planes, 출력 채널: out_planes, 3x3 커널, padding=1로 크기 유지, stride로 크기 조절
        self.bn1 = nn.BatchNorm2d(out_planes)   # 배치 정규화
        self.relu = nn.ReLU()   # 비선형 활성화 함수

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        # 두번째 conv, 채널 수 유지, 크기 유지
        self.bn2 = nn.BatchNorm2d(out_planes)   # 배치 정규화

        self.cbam = CBAM(out_planes) if use_cbam else None  # CBAM 모듈 사용 여부
        self.downsample = downsample    # residual 연결 시 차원 맞추는 conv

    def forward(self, x):
        residual = x    # skip connection용 입력 저장

        out = self.conv1(x) # 첫 번째 conv
        out = self.bn1(out) # 정규화
        out = self.relu(out)  # 활성화

        out = self.conv2(out)   # 두 번째 conv
        out = self.bn2(out) # 정규화

        if self.cbam:
            out = self.cbam(out)    # CBAM 적용

        if self.downsample:
            residual = self.downsample(x)   # shortcut 경로 보정

        out += residual # skip connection
        out = self.relu(out)    # 출력에 ReLU 적용

        return out  # 결과 반환

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64 # 조기 입력 채널 수 설정

        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # 입력: [B, 1, 224, 224] -> 출력: [B, 64, 112, 112], 큰 커널로 넓은 영역 캡처 
        self.bn1 = nn.BatchNorm2d(64)   # 정규화
        self.relu = nn.ReLU()   # 활성화

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # 풀링: [B, 64, 112, 112] -> [B, 64, 56, 56]

        self.layer1 = self._make_layer(64, blocks=2)  # [B, 64, 56, 56] 유지
        self.layer2 = self._make_layer(128, blocks=2, stride=2)  # [B, 128, 28, 28] 다운샘플링
        self.layer3 = self._make_layer(256, blocks=2, stride=2)  # [B, 256, 14, 14] 다운샘플링
        self.layer4 = self._make_layer(512, blocks=2, stride=2, use_cbam=False)  # [B, 512, 7, 7], CBAM 미사용

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # [B, 512, 7, 7] -> [B, 512, 1, 1]
        self.fc = nn.Linear(512, num_classes)  # [B, 512] -> [B, num_classes]

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        # Planes : 해당 레이어의 출력 채널 수
        # # blocks : 블록 수
        # stride=2인 경우 다운샘플링 (해상도 절반)

        downsample = None   # 스킵 연결해서 입력/출력 크기가 다르면 맞춰야 함

        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                # 1x1 conv로 채널 수   및 공간 크기 맞춤
                nn.BatchNorm2d(planes))
            
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        # 첫 블록은 다운샘플링 적용 가능성 있음
        self.in_planes = planes # 이후 블록을 위한 입력 채널 업데이트

        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
            # 나머지 블록은 stride=1로 동일한 해상도 유지

        return nn.Sequential(*layers)   # 블록들을 Seguential로 묶어 반환

    def forward(self, x):
        x = self.conv1(x)  # 입력: [B, 1, 224, 224] -> [B, 64, 112, 112]
        x = self.bn1(x)    # 정규화
        x = self.relu(x)   # ReLU 활성화
        x = self.maxpool(x)  # [B, 64, 112, 112] -> [B, 64, 56, 56]

        x = self.layer1(x)  # [B, 64, 56, 56]
        x = self.layer2(x)  # [B, 128, 28, 28]
        x = self.layer3(x)  # [B, 256, 14, 14]
        x = self.layer4(x)  # [B, 512, 7, 7]

        x = self.avgpool(x)  # [B, 512, 1, 1]
        x = torch.flatten(x, 1)  # [B, 512]
        x = self.fc(x)  # [B, num_classes]

        return x


# -------------------- 학습 루프 --------------------
def run():
    # 모든 CT 슬라이스 파일 경로 불러오기 (LIDC-IDRI 환자 폴더 안의 .npy 파일들)
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))

    # 파일 경로와 해당 파일의 라벨을 튜플로 저장
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    # 라벨이 None이 아닌 데이터만 필터링 (중립 제외)
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]

    # 파일, 라벨을 리스트로 분리
    files, labels = zip(*file_label_pairs)

    # 전체 데이터를 train(70%), val(15%), test(15%)로 분할
    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    # 데이터 불러오기
    train_dataset = CTDataset(train_files, train_labels, transform=train_transform)
    val_dataset = CTDataset(val_files, val_labels, transform=val_transform)
    test_dataset = CTDataset(test_files, test_labels, transform=val_transform)

    # 데이터 로더
    train_loader = DataLoader(train_dataset,  batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # 모델, 손실함수, 옵티마이저 정의
    model = ResNet18_CBAM().to(device)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.6653, 0.3347], device=device))
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0  # 가장 높은 val accuracy를 저장
    save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "r18_cbam_mga_aug_lr4_ep100_weight.pth")

    # 학습 루프 시작
    for epoch in range(num_epochs):
        # MGA 스케쥴링: 초기 lambda -> 점점 증가시킴
        lambda_mga = initial_lambda + (final_lambda - initial_lambda) * (epoch / total_epochs)

        model.train()  # 학습 모드로 변경
        epoch_loss = 0
        correct = 0
        total = 0

        # 한 epoch 동안 모든 train 데이터를 학습
        for images, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images = images.to(device)
            labels = labels.to(device)
            masks = masks.to(device)

            outputs = model(images)  # forward pass
            ce_loss = criterion(outputs, labels)  # cross entropy loss

            # -------------------- MGA Loss 계산 위치 --------------------
            attn_map = model.layer3[1].cbam.last_attention  # attention map 꺼내오기

            if attn_map is not None:
                attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)
                attn_loss = F.mse_loss(attn_map, masks)  # mask와의 MSE loss
                loss = ce_loss + lambda_mga * attn_loss
            else:
                loss = ce_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()

        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")
        print(f"[Epoch {epoch+1}] lambda_mga: {lambda_mga:.4f}")

        torch.cuda.empty_cache(); gc.collect()  # 메모리 정리

        # -------------------- 검증 --------------------
        model.eval()
        correct = 0; total = 0

        with torch.no_grad():
            for iamegs, labels, masks in val_loader:
                iamegs, labels, masks = iamegs.to(device), labels.to(device), masks.to(device)
                outputs = model(iamegs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # -------------------- 테스트 --------------------
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()

    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for images, labels, _ in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    # numpy 배열로 변환
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_probs = np.array(y_probs)

    # 지표 계산
    from sklearn.metrics import (
        classification_report, roc_auc_score, confusion_matrix,
        precision_score, recall_score, balanced_accuracy_score,
        matthews_corrcoef, f1_score
    )

    acc = (y_pred == y_true).mean()
    auc = roc_auc_score(y_true, y_probs)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    specificity = tn / (tn + fp + 1e-6)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    # 📋 출력
    print(f"✅ Test Accuracy         : {acc*100:.2f}%")
    print(f"🎯 AUC                   : {auc:.4f}")
    print(f"📌 Precision             : {precision:.4f}")
    print(f"📌 Recall (Sensitivity)  : {recall:.4f}")
    print(f"📌 Specificity           : {specificity:.4f}")
    print(f"📌 F1 Score              : {f1:.4f}")
    print(f"📌 Balanced Accuracy     : {balanced_acc:.4f}")
    print(f"📌 MCC                   : {mcc:.4f}")
    print("\n📌 Confusion Matrix:")
    print(cm)

    # 📁 CSV로 저장
    test_metrics = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "model": "ResNet18_CBAM_MGA",
        "phase": "test",
        "accuracy": round(acc, 4),
        "auc": round(auc, 4),
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "specificity": round(specificity, 4),
        "f1_score": round(f1, 4),
        "balanced_acc": round(balanced_acc, 4),
        "mcc": round(mcc, 4),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }

    csv_path = "logs/final_test_metrics.csv"
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    file_exists = os.path.exists(csv_path)

    with open(csv_path, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=test_metrics.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(test_metrics)

    print(f"\n📁 테스트 지표 저장 완료: {csv_path}")

# 진입점
if __name__ == "__main__":
    run()


Using device: cuda


[Epoch 1]: 100%|██████████| 467/467 [00:16<00:00, 28.85it/s]


Train Acc: 59.5391, Loss: 0.7025
[Epoch 1] lambda_mga: 0.1000
Val Acc: 0.6112
✅ Saved best model!


[Epoch 2]: 100%|██████████| 467/467 [00:16<00:00, 28.36it/s]


Train Acc: 61.5488, Loss: 0.6686
[Epoch 2] lambda_mga: 0.1040
Val Acc: 0.6850
✅ Saved best model!


...

Train Acc: 97.7492, Loss: 0.0615
[Epoch 98] lambda_mga: 0.4880
Val Acc: 0.8700


[Epoch 99]: 100%|██████████| 467/467 [00:15<00:00, 30.29it/s]


Train Acc: 97.9368, Loss: 0.0599
[Epoch 99] lambda_mga: 0.4920
Val Acc: 0.8850


[Epoch 100]: 100%|██████████| 467/467 [00:11<00:00, 41.58it/s]


Train Acc: 98.1511, Loss: 0.0582
[Epoch 100] lambda_mga: 0.4960
Val Acc: 0.8725

📊 Test Evaluation:
✅ Test Accuracy         : 90.62%
🎯 AUC                   : 0.9408
📌 Precision             : 0.9261
📌 Recall (Sensitivity)  : 0.9314
📌 Specificity           : 0.8582
📌 F1 Score              : 0.9288
📌 Balanced Accuracy     : 0.8948
📌 MCC                   : 0.7917

📌 Confusion Matrix:
[[236  39]
 [ 36 489]]

📁 테스트 지표 저장 완료: logs/final_test_metrics.csv


In [1]:
from glob import glob
import os
import pandas as pd

# 📂 전체 슬라이스 파일 경로 가져오기
slice_root = "/data1/lidc-idri/slices"
all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))

# 🏷️ 파일 이름에서 라벨 추출 함수
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        if score == 3:
            return None  # 중립은 제외
        return int(score >= 4)  # 0 or 1
    except:
        return None

# 📊 라벨 카운트
label_counts = {"0": 0, "1": 0, "None": 0}
for f in all_files:
    label = extract_label_from_filename(f)
    if label is None:
        label_counts["None"] += 1
    else:
        label_counts[str(label)] += 1

# 출력
print("✅ 라벨 분포")
for k, v in label_counts.items():
    print(f"Class {k}: {v}개")

✅ 라벨 분포
Class 0: 1784개
Class 1: 3548개
Class None: 2517개


In [ ]:
# 👍👍👍 전체코드: ResNet18 + CBAM + MGA Loss + Lambda Scheduling (기본) 
# + 데이터 증강 (Resize 224 / RandomHorizontalFlip / RandomRotation 10 / RandomErasing)

import os, re, numpy as np, torch, gc
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import pandas as pd
import cv2
import torchvision.transforms as transforms
from PIL import Image

# -------------------- 디바이스 설정 --------------------
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------- 하이퍼파라미터 설정 --------------------
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"

batch_size = 16
num_epochs = 100
learning_rate = 1e-4

# lambda MGA 스케줄 설정
initial_lambda = 0.1
final_lambda = 0.5
total_epochs = num_epochs

# -------------------- Transform --------------------
train_transform = transforms.Compose([
    transforms.ToPILImage(),    # numpy or tensor 이미지를 PIL 이미지 객체로 변환
    transforms.Resize((224, 224)),  # 이미지를 224x224로 resize
    transforms.RandomHorizontalFlip(),  # 이미지를 50% 확률로 좌우 반전
    transforms.RandomRotation(10),  # 이미지를 -10도 ~ +10도 사이로 랜덤 회전, 촬영 자세나 기울어짐에 대한 회전 강건성확보
    transforms.ToTensor(),  # PIL이미지 -> PyTorch Tensor로 변환, (H, W, C) -> (C, H, W), 값도 0255 -> 01 사이즈로 스케일 조정
    transforms.Normalize([0.5], [0.5]), # 평균 0.5, 표준편차 0.5로 정규화 -> 결과적으로 01 -> 11로 바뀜
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])  # 전체 이미지의 일부분 지우고 0으로 채움 (검은 사각형 생김)
    # p=0.5 : 50% 확률로 이 증강 적용
    # scale : 전체 이미지 대비 삭제 영역의 크기 비율
    # ratio : 지우는 사각형의 가로:세로 비율 범위
    # value=0 : 지운 곳을 검은색(0)으로 덮음
    # 폐 CT에서 병변이 항상 일정한 위치에 나오지 않으니까 모델이 특정 위치에 과적합되는걸 방지함 (overfitting 예방)

# 검증/테스트는 모델이 학습하지 않은 깨긋한 상태의 이미지로 정확도를 확인하기 위해서 검증용은 깔끔하게
val_transform = transforms.Compose([
    transforms.ToPILImage(),    # PIL 이미지 객체로 변환
    transforms.Resize((224, 224)),  # 사이즈 맞추기
    transforms.ToTensor(),  # PIL 이미지 -> PyTorch Tensor로 변환
    transforms.Normalize([0.5], [0.5])  # 정규화
])

# -------------------- Bounding Box를 Binary Mask로 --------------------
def create_binary_mask_from_bbox(bbox_list, image_size=(224, 224)):
    # bbox_list : 한 이미지에 들어있는 bounding box 리스트
    # image_size : 출력할 마스크 크기. 보통 이미지와 동일한 (height, width) -> 디폴트는 224x224
    # bbox들을 binary mask로 바꿔주는 함수
    masks = []  # 여러 개의 bbox가 들어오니까, 각각의 마스크를 하나씩 리스트에 쌓기 위한 빈 리스트
    for bbox in bbox_list:  # bbox_list를 하나씩 돌면서 처리 -> [x_min, y_min, x_max, y_max]네 좌표로 구성된 하나의 사각형 영역 
        mask = np.zeros(image_size, dtype=np.float32)   # 224x224짜리 0으로 꽉 찬 2D 배열을 하나 생성
        # 배경이 흰 종이를 만드는 느낌으로 만들고, 사각 영역만 1로 덧칠할거임
        x_min, y_min, x_max, y_max = bbox   # 각 bbox의 네 좌표값을 각각 변수로 언팩. -> 마스크의 해당 영역에 사각형을 칠하기 위해서
        mask[y_min:y_max, x_min:x_max] = 1.0    # y_min, y_max, x_min, x_max까지의 범위에 1.0을 채워 넣음
        # -> 마스크에서 bbox에 해당하는 사각형 영역만 1(foreground)로 표시됨. 나머진 여전히 0(background)
        masks.append(mask)  # 지금 만든 마스크(2D 배열)를 리스트에 추가 -> [mask1, mask2, ...]이렇게 쌓임

    masks = np.stack(masks) # 리스트를 하나의 3D 배열로 합침 -> shape : [N, H, W] -> N은 bbox 개수
    masks = np.expand_dims(masks, axis=1)   # 텐서 shape을 [N, 1, H, W]로 바꿈
    # PyTorch 모델에서 기대하는 (batch x channel x height x width) 포맷 맞추기

    return torch.tensor(masks, dtype=torch.float32)
    # numpy 배열을 PyTorch 텐서로 변환해서 리턴

    # 한 bbox → 하나의 마스크 → 여러 개면 쌓아서 batch 형태로
# -------------------- Bounding Box CSV 로드 --------------------
def load_bbox_dict(csv_path):
    # csv_path : bounding box 정보가 들어있는 CSV 파일 경로
    # 반환값 : {filename:[bbox1, bbox2, ...]} 형태의 딕셔너리
    df = pd.read_csv(csv_path)  # CSV파일을 pandas DataFrame으로 읽어옴
    bbox_dict = {}
    # key : 슬라이스 파일 이름 (ex. "LIDC-IDRI-1012_slice0004.npy")
    # value : 해당 슬라이스에 존재하는 bbox들의 리스트
    for _, row in df.iterrows():    # DataFrame의 모든 행(row)를 하나씩 순회
        # row는 한 줄(=한 bbox)의 정보를 담고 있음

        pid = row['pid']    # 환자 ID (예: "LIDC-IDRI-1012") -> 이미지 이름 구성 요소
        slice_str = row['slice']    # 슬라이스 정보가 들어있는 문자열 (예: "slice_0039")
        slice_idx = int(re.findall(r'\d+', str(slice_str))[0])  # re.findall()로 문자열에서 숫자만 뽑아냄
        # "slice_0039" -> ['0039'] -> [0] -> 39 (슬라이스 번호를 정수로 추출함)
        fname = f"{pid}_slice{slice_idx:04d}.npy"   # 파일명 구성 (예: "LIDC-IDRI-1012_slice0039.npy")
        # {:04d}는 4자리 정수로 만들고 빈자리는 0으로 채워줌 (39 -> 0039)
        bbox = eval(row['bb'])  # row['bb']는 문자열 형태의 bbox (예: "[20, 30, 80, 100]")
        # eval()을 써서 문자열을 리스트로 바꿔줌
        # 주의 : 보안 상 위험할 수 있는 함수지만, 여긴 내부 데이터라 사용중
        bbox_dict.setdefault(fname, []).append(bbox)    # fname이라는 key가 딕셔너리에 없으면 []로 초기화하고,
        # 거기에 bbox를 append -> 슬라이스 하나에 bbox 여러개 있어도 전부 리스트로 모아줌
    return bbox_dict    # 최종적으로 {filename: [bbox1, bbox2, ...]} 형태의 딕셔너리 반환

bbox_dict = load_bbox_dict(bbox_csv_path)
# 실제로 csv_path에 있는 정보를 불러와서 bbox_dict에 저장함
# 이걸 나주에 Dataset 클래스에서 fname 기준을 꺼내쓰게 됨

# -------------------- 라벨 추출 --------------------
def extract_label_from_filename(fname): # fname : 파일 이름 (예: "LIDC-IDRI-1012_slice0039_5.npy")
    # 이 이름에서 malignancy score(악성도 점수)를 추출해서 라벨로 변환
    try:    # 파일명이 이상하거나 에러나면 except로 빠져나가서 None 반환함 (안전장치)
        score = int(fname.split("_")[-1].replace(".npy", ""))
        # 파일명에서 _ 제외하고 나머지 것들 중에 마지막에꺼를 가져와서 .npy를 "" 이렇게 공백으로 처리함
        # fname.split("_") -> ['LIDC-IDRI-1012', 'slice0039', '5.npy]
        # [-1] -> '5.npy'
        # .replace(".npy", "") -> '5'
        # int(...) -> 5 <- 이게 malignancy score
        return None if score == 3 else int(score >= 4)
        # 라벨 결정 로직으로
        # score == 3 -> 중립 -> None 반환 -> 학습에서 제외
        # score >= 4 -> 암(양성) -> 1
        # score <= 2 -> 정상(음성) -> 0
        # int(score >= 4)는 파이썬에서 True -> 1
        # False -> 0 이니깐 자동으로 라벨이 됨
    except:
        return None
        # 혹시 split이나 replace, int 변환이 실패하면 그냥 None 반환하고 무시

# -------------------- Dataset --------------------
class CTDataset(Dataset):
    # PyTorch의 Dataset 클래스를 상속해서 커ㅡ텀 데이터셋 정의
    # 나중에 DataLoader랑 같이 쓰이기 때문에 __len__()이랑 __getitem__()을 꼭 넣어줘야함
    def __init__(self, paths, labels, transform=None):  # 생성자 : 세개의 인자를 받음
        # paths : 이미지 .npy 파일 경로 리스트
        # labels : 각 이미지에 대한 라벨 리스트 (0, 1 or None)
        # transform : 이미지 증강 설정 (train_transform, val_transform 등)
        self.paths = paths
        self.labels = labels
        self.transform = transform
        # 받은 인자를 멤버 변수로 저장. 나중에 gettem()에서 접근함

    def __getitem__(self, idx): # DataLoader가 이걸 호출할 때 index에 해당하는 sample 하나를 반환
        # 이미지, 라벨, 마스크( = MGA용 target) 3개를 리턴함
        file_path = self.paths[idx] # 파일 경로 불러오기
        label = self.labels[idx]    # 라벨 불러오기
        fname = os.path.basename(file_path) # 전체 경로에서 파일 이름만 추출 -> 나중에 bbox_dict[fname] 찾을때 쓰임

        img = np.load(file_path)    # .npy 파일에서 CT 슬라이스 불러오기 -> 흑백 CT 이미지, shape은 (H, W)
        img = np.clip(img, -1000, 400)  # CT 이미지 HU 값이 너무 크거나 작으면 노이즈 -> -1000(공기) ~ 400(연조직)으로 클리핑해서 노이즈 제거
        img = (img + 1000) / 1400.  # 정규화 : -1000 -> 0, 400 -> 1 사이 값으로 바꿔줌 -> 모델이 안정적으로 학습할 수 있도록 함
        img = np.expand_dims(img, axis=-1)  # CT는 채널이 1개니깐 (H, W) -> (H, w, 1)로 바꿔줌
        # 나중에 PyTorch에서 (C, H, W)로 바꾸기 위함

        if self.transform:  # 데이터 증강(transform)이 있다면 적용
            img = self.transform(img)   
        else:   # 없으면 numpy -> tensor 변환하고 (H, W, C) -> (C, H, W)로 순서 바꿈
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)

        if fname in bbox_dict:  # 이 이미지에 bbox가 존재하면 -> 마스크 생성
            mask = create_binary_mask_from_bbox(bbox_dict[fname], image_size=(224, 224))
            # image_size는 transform과 동일하게 224x224
        else:   # bbox가 없다면 전부 0으로 채워진 마스크 생성 -> MGA Loss 계산 시 참고용으로 쓰일 수 있음
            mask = torch.zeros((1, 224, 224), dtype=torch.float32)

        return img, torch.tensor(label).long(), mask.squeeze(0)
        # 반환값 3개 :
        # img : shape[1, 224, 224]
        # label : int(0 or 1)
        # mask : [224, 224] <- squeeze로 채널 1개 제거

    def __len__(self):
        return len(self.paths)
    # 전체 데이터셋 길이 반환 -> DataLoader가 아라야 배치 쪼갤 수 있음.

# -------------------- CBAM 정의 (MGA 포함) --------------------
# 2 Step : Channel Attention(어떤 채널에 집중할지) * Spatial Attention(어디에 집중할지) = 최종 Attention

class ChannelAttention(nn.Module):  # 입력 feature map의 채널별 중요도를 계산해서 강조함
    def __init__(self, planes, ratio=16):
        # planes : 입력 채널 수
        # ratio : 중간 채널 축소 비율. 기본 1/16으로 bottlenck 구성
        super().__init__()

        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        # MLP 역할을 하는 1x1 conv 블록 -> 채널 압축 -> 비선형 -> 복원 (shared는 avg/max 둘다에서 같이 씀)

        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()
        # 평균 풀링 / 최대 풀링으로 두가지 전역 정보를 추출
        # 마지막 sigmoid는 attention weight로 스케일링

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))
    # avg & max 풀링 경과를 각각 shape MLP에 통과시키고, 더한 후 sigmoid
    # -> shape : [B, C, 1, 1]
    # -> 채널마다 중요도 weight를 곱하게 됨

class SpatialAttention(nn.Module):  # 공간적으로 어디에 집중할지를 결정 -> 각 채널 내부에서 중요한 위치 찾기

    def __init__(self, k=7):    
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    # 채널 차원은 평균, 최대 두 개만 써서 concat
    # 그걸 1채널로 줄여주는 conv
    # 커널 크기 k=7이면 넓은 영역까지 감지 가능

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))
    # 입력 feature map에서 :
    # 평균, 최대값을 각 spatial 위치별로 구함 -> [B, 1, H, W] 두 개
    # concat -> [B, 2, H, w]
    # conv + sigmoid -> 위치별 중요도 map

class CBAM(nn.Module):  
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()
        self.last_attention = None
    # ChannelAttention, SpatialAttention을 내부에 선언
    # MGA를 위해 마지막 attention map을 저장하는 변수 포함

    def forward(self, x):
        ca_out = self.ca(x) * x
        sa_out = self.sa(ca_out)
        self.last_attention = sa_out
        return sa_out * ca_out
    # 채널 중요도 -> 곱함
    # 위치 중요도 -> 곱함
    # 둘 다 반영된 최종 feature map 리턴

# -------------------- ResNet18 + CBAM 모델 정의 --------------------
# BasicBlockCBAM : ResNet의 기본 Residual Block 하나를 정의
# → conv → BN → ReLU → conv → BN → (CBAM optional) → Add → ReLU

# ResNet18_CBAM : ResNet18 구조로 전체 네트워크 쌓기
# → conv1 → layer1~3 → layer4 → avgpool → fc

class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        # 입력 채널: in_planes, 출력 채널: out_planes, 3x3 커널, padding=1로 크기 유지, stride로 크기 조절
        self.bn1 = nn.BatchNorm2d(out_planes)   # 배치 정규화
        self.relu = nn.ReLU()   # 비선형 활성화 함수

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        # 두번째 conv, 채널 수 유지, 크기 유지
        self.bn2 = nn.BatchNorm2d(out_planes)   # 배치 정규화

        self.cbam = CBAM(out_planes) if use_cbam else None  # CBAM 모듈 사용 여부
        self.downsample = downsample    # residual 연결 시 차원 맞추는 conv

    def forward(self, x):
        residual = x    # skip connection용 입력 저장

        out = self.conv1(x) # 첫 번째 conv
        out = self.bn1(out) # 정규화
        out = self.relu(out)  # 활성화

        out = self.conv2(out)   # 두 번째 conv
        out = self.bn2(out) # 정규화

        if self.cbam:
            out = self.cbam(out)    # CBAM 적용

        if self.downsample:
            residual = self.downsample(x)   # shortcut 경로 보정

        out += residual # skip connection
        out = self.relu(out)    # 출력에 ReLU 적용

        return out  # 결과 반환

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64 # 조기 입력 채널 수 설정

        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # 입력: [B, 1, 224, 224] -> 출력: [B, 64, 112, 112], 큰 커널로 넓은 영역 캡처 
        self.bn1 = nn.BatchNorm2d(64)   # 정규화
        self.relu = nn.ReLU()   # 활성화

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # 풀링: [B, 64, 112, 112] -> [B, 64, 56, 56]

        self.layer1 = self._make_layer(64, blocks=2)  # [B, 64, 56, 56] 유지
        self.layer2 = self._make_layer(128, blocks=2, stride=2)  # [B, 128, 28, 28] 다운샘플링
        self.layer3 = self._make_layer(256, blocks=2, stride=2)  # [B, 256, 14, 14] 다운샘플링
        self.layer4 = self._make_layer(512, blocks=2, stride=2, use_cbam=False)  # [B, 512, 7, 7], CBAM 미사용

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # [B, 512, 7, 7] -> [B, 512, 1, 1]
        self.fc = nn.Linear(512, num_classes)  # [B, 512] -> [B, num_classes]

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        # Planes : 해당 레이어의 출력 채널 수
        # # blocks : 블록 수
        # stride=2인 경우 다운샘플링 (해상도 절반)

        downsample = None   # 스킵 연결해서 입력/출력 크기가 다르면 맞춰야 함

        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                # 1x1 conv로 채널 수   및 공간 크기 맞춤
                nn.BatchNorm2d(planes))
            
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        # 첫 블록은 다운샘플링 적용 가능성 있음
        self.in_planes = planes # 이후 블록을 위한 입력 채널 업데이트

        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
            # 나머지 블록은 stride=1로 동일한 해상도 유지

        return nn.Sequential(*layers)   # 블록들을 Seguential로 묶어 반환

    def forward(self, x):
        x = self.conv1(x)  # 입력: [B, 1, 224, 224] -> [B, 64, 112, 112]
        x = self.bn1(x)    # 정규화
        x = self.relu(x)   # ReLU 활성화
        x = self.maxpool(x)  # [B, 64, 112, 112] -> [B, 64, 56, 56]

        x = self.layer1(x)  # [B, 64, 56, 56]
        x = self.layer2(x)  # [B, 128, 28, 28]
        x = self.layer3(x)  # [B, 256, 14, 14]
        x = self.layer4(x)  # [B, 512, 7, 7]

        x = self.avgpool(x)  # [B, 512, 1, 1]
        x = torch.flatten(x, 1)  # [B, 512]
        x = self.fc(x)  # [B, num_classes]

        return x


# -------------------- 학습 루프 --------------------
def run():
    # 모든 CT 슬라이스 파일 경로 불러오기 (LIDC-IDRI 환자 폴더 안의 .npy 파일들)
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))

    # 파일 경로와 해당 파일의 라벨을 튜플로 저장
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    # 라벨이 None이 아닌 데이터만 필터링 (중립 제외)
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]

    # 파일, 라벨을 리스트로 분리
    files, labels = zip(*file_label_pairs)

    # 전체 데이터를 train(70%), val(15%), test(15%)로 분할
    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    # 데이터 불러오기
    train_dataset = CTDataset(train_files, train_labels, transform=train_transform)
    val_dataset = CTDataset(val_files, val_labels, transform=val_transform)
    test_dataset = CTDataset(test_files, test_labels, transform=val_transform)

    # 데이터 로더
    train_loader = DataLoader(train_dataset,  batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # 모델, 손실함수, 옵티마이저 정의
    model = ResNet18_CBAM().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc = 0.0  # 가장 높은 val accuracy를 저장
    save_path = os.path.join(os.path.dirname(os.getcwd()), "pth", "r18_cbam_mga_aug_lr4_ep100_1t.pth")

    # 학습 루프 시작
    for epoch in range(num_epochs):
        # MGA 스케쥴링: 초기 lambda -> 점점 증가시킴
        lambda_mga = initial_lambda + (final_lambda - initial_lambda) * (epoch / total_epochs)

        model.train()  # 학습 모드로 변경
        epoch_loss = 0
        correct = 0
        total = 0

        # 한 epoch 동안 모든 train 데이터를 학습
        for images, labels, masks in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            images = images.to(device)
            labels = labels.to(device)
            masks = masks.to(device)

            outputs = model(images)  # forward pass
            ce_loss = criterion(outputs, labels)  # cross entropy loss

            # -------------------- MGA Loss 계산 위치 --------------------
            attn_map = model.layer3[1].cbam.last_attention  # attention map 꺼내오기

            if attn_map is not None:
                attn_map = F.interpolate(attn_map, size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)
                attn_loss = F.mse_loss(attn_map, masks)  # mask와의 MSE loss
                loss = ce_loss + lambda_mga * attn_loss
            else:
                loss = ce_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            epoch_loss += loss.item()

        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")
        print(f"[Epoch {epoch+1}] lambda_mga: {lambda_mga:.4f}")

        torch.cuda.empty_cache(); gc.collect()  # 메모리 정리

        # -------------------- 검증 --------------------
        model.eval()
        correct = 0; total = 0

        with torch.no_grad():
            for iamegs, labels, masks in val_loader:
                iamegs, labels, masks = iamegs.to(device), labels.to(device), masks.to(device)
                outputs = model(iamegs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # -------------------- 테스트 --------------------
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path))
    model.eval()
    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for iamegs, labels, masks in test_loader:
            iamegs, labels, masks = iamegs.to(device), labels.to(device), masks.to(device)
            outputs = model(iamegs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    # 최종 테스트 결과 출력
    print(f"✅ Test Accuracy: {(np.array(y_pred) == np.array(y_true)).mean() * 100:.2f}%")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"AUC: {roc_auc_score(y_true, y_probs):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

# 진입점
if __name__ == "__main__":
    run()
